## Slicing, combining, and exporting water column features for O2-based productivity estimates

### Upstream:

#### - Started with using python-CTD to convert 1 m binned CTD files (Seabird cnv) to a single csv for each station 
#### - Downloaded Al's 'ave' cnv files from the POMZ shared drive

### Downstream:

#### - Will take times and O2 concentrations to estimate GOP and CR in the mixed layer, the primary chlorophyll max, and the deep chlorophyll max using MATLAB sine fitting script from Barone et al., 2019.

### Station 20/P2, October 2019 (Kilo Moana, KM1919):

in 2019_St20_P2_density, we saw that the deep chl max stays in the 25.7-26.2 range. So we'll sum first the chl in that range to look at chl over time during the station occupation (October 03-13).

In [1]:
import io
import requests
from pathlib import Path
import seaborn as sns
import pandas as pd

import ctd
import datetime as dt

In [2]:
# Little script to convert datetime to MATLAB datenum format to go into Barone ML script later

def datetime_to_datenum(dt_obj):
    matlab_datenum = dt_obj.toordinal() + 366 + (dt_obj - dt.datetime(2000, 1, 1)).total_seconds() / 86400
    return matlab_datenum

In [3]:
# Test how the datenum conversion works
now = dt.datetime.now()  # get current datetime
matlab_datenum = datetime_to_datenum(now)
print(matlab_datenum)
print(now)

747490.904002998
2023-04-12 21:41:45.859026


In [4]:
# Define the starting datetimes of each cast

st25 = '2019-10-03 13:31:35'
st28 = '2019-10-04 11:01:39'
st29 = '2019-10-04 23:50:04'
st30 = '2019-10-05 08:39:43'
st31 = '2019-10-05 14:03:02'
st32 = '2019-10-05 21:38:22'
st33 = '2019-10-06 14:14:26'
st34 = '2019-10-06 19:38:21'
st35 = '2019-10-06 22:48:57'
st36 = '2019-10-07 01:47:02'
st37 = '2019-10-07 07:40:03'
st38 = '2019-10-07 16:50:08'
st39 = '2019-10-07 20:45:29'
st40 = '2019-10-08 01:04:04'
st42 = '2019-10-08 07:04:45'
st43 = '2019-10-08 13:20:29'
st44 = '2019-10-08 15:59:43'
st45 = '2019-10-08 18:55:51'
st46 = '2019-10-08 22:10:38'
st47 = '2019-10-09 01:10:36'
st48 = '2019-10-09 07:47:50'
st49 = '2019-10-09 11:32:27'
st50 = '2019-10-09 14:15:42'
st51 = '2019-10-09 19:13:01'
st52 = '2019-10-11 00:09:58'
st53 = '2019-10-11 04:49:54'
st54 = '2019-10-11 12:24:17'
st55 = '2019-10-11 19:08:20'
st56 = '2019-10-11 23:05:27'
st57 = '2019-10-12 02:30:37'
st58 = '2019-10-12 10:31:29'
st59 = '2019-10-12 13:13:32'
st60 = '2019-10-12 19:49:39'
st61 = '2019-10-13 05:08:47'

### Cell structure

1.  Read from the individual cast cnv
2.  Add the correct and pertinent start time to each downcast
3.  Slice the mixed layer and save as a csv in KM1919/P2_2019/mixed-layer/
4.  Integrate the O2, beam transmission, and chlorophyll fluoresence over the mixed layer and save each as a variable
5.  save a new dataframe with the lat, long, StartTime, and integrated value variables

6.  Repeat steps 3-5 for the PCM and DCM

7. Combine all cast integration dataframes for each water column feature and save as a csv in KM1919/P2_2019/integrations/

In [5]:
# read from cnv with 'ignore' added to depSM
cast25 = ctd.from_cnv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/KM1919-St20-P2/St20_P2_cast25-align-cellTM-derive-bin.cnv")

# set up datetime to datenum 
ST_25 = pd.to_datetime(st25)

# split up and down casts; we want the downcast for the profile
down25, up25 = cast25.split()

# add a column with the start date and time in UTC (this is from the cnv header or from ship's log)
down25.insert(0, 'StartTime', pd.to_datetime(st25).replace(microsecond=0))

# slice the mixed layer out and save as csv
p2_25_ml = down25[down25['sigma-�00'].between(22.0,23.2)]
p2_25_ml.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/mixed_layer//Station20_cast25_ML.csv")

# get the sum of mixed layer slice oxygen, beam transmission, and chlorophyll fluor
cf_sum_p2_25_ml = p2_25_ml['flECO-AFL'].sum()
print('ML Chl fluor sum:', cf_sum_p2_25_ml)

bt_sum_p2_25_ml = p2_25_ml['CStarTr0'].sum()
print('ML Beam trans sum:', bt_sum_p2_25_ml)

o2_sum_p2_25_ml = p2_25_ml['sbeox0Mm/L'].sum()
print('ML O2 Mm/Kg sum:', o2_sum_p2_25_ml)

# Make lists of the input for the ML integrated dataframe
ML_data_25 = {'Cast':[25], 'StartTime':st25, 'StartTime matlab datenum': datetime_to_datenum(ST_25), \
           'Int chl fluor': cf_sum_p2_25_ml, \
           'Int beam trans': bt_sum_p2_25_ml, \
            'Int O2 Mm/Kg': o2_sum_p2_25_ml}

# slice the primary chlorophyll max out and save as csv
p2_25_pcm = down25[down25['sigma-�00'].between(23.2,25.7)]
p2_25_pcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/primary_chl/Station20_cast25_PCM.csv")

# get the sum of primary chl slice oxygen and beam transmission
cf_sum_p2_25_pcm = p2_25_pcm['flECO-AFL'].sum()
print('PCM Chl fluor sum:', cf_sum_p2_25_pcm)

bt_sum_p2_25_pcm = p2_25_pcm['CStarTr0'].sum()
print('PCM Beam trans sum:', bt_sum_p2_25_pcm)

o2_sum_p2_25_pcm = p2_25_pcm['sbeox0Mm/L'].sum()
print('PCM O2 Mm/Kg sum:', o2_sum_p2_25_pcm)

# Make lists of the input for the PCM integrated dataframe
PCM_data_25 = {'Cast':[25], 'StartTime':st25, 'StartTime matlab datenum': datetime_to_datenum(ST_25), \
            'Int chl fluor': cf_sum_p2_25_pcm, \
           'Int beam trans': bt_sum_p2_25_pcm, \
            'Int O2 Mm/Kg': o2_sum_p2_25_pcm}

# slice the deep chlorophyll max out and save as csv
p2_25_dcm = down25[down25['sigma-�00'].between(25.7,26.2)]
p2_25_dcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/deep_chl/Station20_cast25_DCM.csv")

# get the sum of deep chl slice oxygen and beam transmission
cf_sum_p2_25_dcm = p2_25_dcm['flECO-AFL'].sum()
print('DCM Chl fluor sum:', cf_sum_p2_25_dcm)

bt_sum_p2_25_dcm = p2_25_dcm['CStarTr0'].sum()
print('DCM Beam trans sum:', bt_sum_p2_25_dcm)

o2_sum_p2_25_dcm = p2_25_dcm['sbeox0Mm/L'].sum()
print('DCM O2 Mm/Kg sum:', o2_sum_p2_25_dcm)

# Make lists of the input for the DCM integrated dataframe
DCM_data_25 = {'Cast':[25], 'StartTime':st25, 'StartTime matlab datenum': datetime_to_datenum(ST_25), \
            'Int chl fluor': cf_sum_p2_25_dcm, \
           'Int beam trans': bt_sum_p2_25_dcm, \
            'Int O2 Mm/Kg': o2_sum_p2_25_dcm}

ML Chl fluor sum: 4.4364
ML Beam trans sum: 1279.5396999999998
ML O2 Mm/Kg sum: 2828.7250000000004
PCM Chl fluor sum: 24.7593
PCM Beam trans sum: 4051.8445
PCM O2 Mm/Kg sum: 3690.297
DCM Chl fluor sum: 16.085099999999997
DCM Beam trans sum: 4968.1428
DCM O2 Mm/Kg sum: 77.584


In [6]:
# Make lists into a new dataframe called ML_2019_p2
ML_2019_p2 = pd.DataFrame(ML_data_25)

# Same for PCM
PCM_2019_p2 = pd.DataFrame(PCM_data_25)

# And DCM
DCM_2019_p2 = pd.DataFrame(DCM_data_25)

DCM_2019_p2.head()

,Cast,StartTime,StartTime matlab datenum,Int chl fluor,Int beam trans,Int O2 Mm/Kg
0,25,2019-10-03 13:31:35,744916.5636,16.0851,4968.1428,77.584


In [7]:
# read from cnv with 'ignore' added to depSM
cast28 = ctd.from_cnv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/KM1919-St20-P2/St20_P2_cast28-align-cellTM-derive-bin.cnv")

# set up datetime to datenum 
ST_28 = pd.to_datetime(st28)

# split up and down casts; we want the downcast for the profile
down28, up28 = cast28.split()

# add a column with the start date and time in UTC (this is from the cnv header or from ship's log)
down28.insert(0, 'StartTime', pd.to_datetime(st28).replace(microsecond=0))

# slice the mixed layer out and save as csv
p2_28_ml = down28[down28['sigma-�00'].between(22.0,23.2)]
p2_28_ml.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/mixed_layer//Station20_cast28_ML.csv")

# get the sum of mixed layer slice oxygen, beam transmission, and chlorophyll fluor
cf_sum_p2_28_ml = p2_28_ml['flECO-AFL'].sum()
print('ML Chl fluor sum:', cf_sum_p2_28_ml)

bt_sum_p2_28_ml = p2_28_ml['CStarTr0'].sum()
print('ML Beam trans sum:', bt_sum_p2_28_ml)

o2_sum_p2_28_ml = p2_28_ml['sbeox0Mm/L'].sum()
print('ML O2 Mm/Kg sum:', o2_sum_p2_28_ml)

# Make lists of the input for the ML integrated dataframe
ML_data_28 = {'Cast':[28], 'StartTime':st28, 'StartTime matlab datenum': datetime_to_datenum(ST_28), \
           'Int chl fluor': cf_sum_p2_28_ml, \
           'Int beam trans': bt_sum_p2_28_ml, \
            'Int O2 Mm/Kg': o2_sum_p2_28_ml}

# slice the primary chlorophyll max out and save as csv
p2_28_pcm = down28[down28['sigma-�00'].between(23.2,28.7)]
p2_28_pcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/primary_chl/Station20_cast28_PCM.csv")

# get the sum of primary chl slice oxygen and beam transmission
cf_sum_p2_28_pcm = p2_28_pcm['flECO-AFL'].sum()
print('PCM Chl fluor sum:', cf_sum_p2_28_pcm)

bt_sum_p2_28_pcm = p2_28_pcm['CStarTr0'].sum()
print('PCM Beam trans sum:', bt_sum_p2_28_pcm)

o2_sum_p2_28_pcm = p2_28_pcm['sbeox0Mm/L'].sum()
print('PCM O2 Mm/Kg sum:', o2_sum_p2_28_pcm)

# Make lists of the input for the PCM integrated dataframe
PCM_data_28 = {'Cast':[28], 'StartTime':st28, 'StartTime matlab datenum': datetime_to_datenum(ST_28), \
            'Int chl fluor': cf_sum_p2_28_pcm, \
           'Int beam trans': bt_sum_p2_28_pcm, \
            'Int O2 Mm/Kg': o2_sum_p2_28_pcm}

# slice the deep chlorophyll max out and save as csv
p2_28_dcm = down28[down28['sigma-�00'].between(28.7,26.2)]
p2_28_dcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/deep_chl/Station20_cast28_DCM.csv")

# get the sum of deep chl slice oxygen and beam transmission
cf_sum_p2_28_dcm = p2_28_dcm['flECO-AFL'].sum()
print('DCM Chl fluor sum:', cf_sum_p2_28_dcm)

bt_sum_p2_28_dcm = p2_28_dcm['CStarTr0'].sum()
print('DCM Beam trans sum:', bt_sum_p2_28_dcm)

o2_sum_p2_28_dcm = p2_28_dcm['sbeox0Mm/L'].sum()
print('DCM O2 Mm/Kg sum:', o2_sum_p2_28_dcm)

# Make lists of the input for the DCM integrated dataframe
DCM_data_28 = {'Cast':[28], 'StartTime':st28, 'StartTime matlab datenum': datetime_to_datenum(ST_28), \
            'Int chl fluor': cf_sum_p2_28_dcm, \
           'Int beam trans': bt_sum_p2_28_dcm, \
            'Int O2 Mm/Kg': o2_sum_p2_28_dcm}

ML Chl fluor sum: 4.1781999999999995
ML Beam trans sum: 1375.1804000000002
ML O2 Mm/Kg sum: 3074.425
PCM Chl fluor sum: 57.709500000000006
PCM Beam trans sum: 26594.2224
PCM O2 Mm/Kg sum: 5969.360000000001
DCM Chl fluor sum: 0.0
DCM Beam trans sum: 0.0
DCM O2 Mm/Kg sum: 0.0


In [8]:
# read from cnv with 'ignore' added to depSM
cast29 = ctd.from_cnv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/KM1919-St20-P2/St20_P2_cast29-align-cellTM-derive-bin.cnv")

# set up datetime to datenum 
ST_29 = pd.to_datetime(st29)

# split up and down casts; we want the downcast for the profile
down29, up29 = cast29.split()

# add a column with the start date and time in UTC (this is from the cnv header or from ship's log)
down29.insert(0, 'StartTime', pd.to_datetime(st29).replace(microsecond=0))

# slice the mixed layer out and save as csv
p2_29_ml = down29[down29['sigma-�00'].between(22.0,23.2)]
p2_29_ml.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/mixed_layer//Station20_cast29_ML.csv")

# get the sum of mixed layer slice oxygen, beam transmission, and chlorophyll fluor
cf_sum_p2_29_ml = p2_29_ml['flECO-AFL'].sum()
print('ML Chl fluor sum:', cf_sum_p2_29_ml)

bt_sum_p2_29_ml = p2_29_ml['CStarTr0'].sum()
print('ML Beam trans sum:', bt_sum_p2_29_ml)

o2_sum_p2_29_ml = p2_29_ml['sbeox0Mm/L'].sum()
print('ML O2 Mm/Kg sum:', o2_sum_p2_29_ml)

# Make lists of the input for the ML integrated dataframe
ML_data_29 = {'Cast':[29], 'StartTime':st29, 'StartTime matlab datenum': datetime_to_datenum(ST_29), \
           'Int chl fluor': cf_sum_p2_29_ml, \
           'Int beam trans': bt_sum_p2_29_ml, \
            'Int O2 Mm/Kg': o2_sum_p2_29_ml}

# slice the primary chlorophyll max out and save as csv
p2_29_pcm = down29[down29['sigma-�00'].between(23.2,29.7)]
p2_29_pcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/primary_chl/Station20_cast29_PCM.csv")

# get the sum of primary chl slice oxygen and beam transmission
cf_sum_p2_29_pcm = p2_29_pcm['flECO-AFL'].sum()
print('PCM Chl fluor sum:', cf_sum_p2_29_pcm)

bt_sum_p2_29_pcm = p2_29_pcm['CStarTr0'].sum()
print('PCM Beam trans sum:', bt_sum_p2_29_pcm)

o2_sum_p2_29_pcm = p2_29_pcm['sbeox0Mm/L'].sum()
print('PCM O2 Mm/Kg sum:', o2_sum_p2_29_pcm)

# Make lists of the input for the PCM integrated dataframe
PCM_data_29 = {'Cast':[29], 'StartTime':st29, 'StartTime matlab datenum': datetime_to_datenum(ST_29), \
            'Int chl fluor': cf_sum_p2_29_pcm, \
           'Int beam trans': bt_sum_p2_29_pcm, \
            'Int O2 Mm/Kg': o2_sum_p2_29_pcm}

# slice the deep chlorophyll max out and save as csv
p2_29_dcm = down29[down29['sigma-�00'].between(29.7,26.2)]
p2_29_dcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/deep_chl/Station20_cast29_DCM.csv")

# get the sum of deep chl slice oxygen and beam transmission
cf_sum_p2_29_dcm = p2_29_dcm['flECO-AFL'].sum()
print('DCM Chl fluor sum:', cf_sum_p2_29_dcm)

bt_sum_p2_29_dcm = p2_29_dcm['CStarTr0'].sum()
print('DCM Beam trans sum:', bt_sum_p2_29_dcm)

o2_sum_p2_29_dcm = p2_29_dcm['sbeox0Mm/L'].sum()
print('DCM O2 Mm/Kg sum:', o2_sum_p2_29_dcm)

# Make lists of the input for the DCM integrated dataframe
DCM_data_29 = {'Cast':[29], 'StartTime':st29, 'StartTime matlab datenum': datetime_to_datenum(ST_29), \
            'Int chl fluor': cf_sum_p2_29_dcm, \
           'Int beam trans': bt_sum_p2_29_dcm, \
            'Int O2 Mm/Kg': o2_sum_p2_29_dcm}

ML Chl fluor sum: 4.4235
ML Beam trans sum: 1172.4138000000003
ML O2 Mm/Kg sum: 2621.004
PCM Chl fluor sum: 66.5312
PCM Beam trans sum: 26773.471599999997
PCM O2 Mm/Kg sum: 5158.425
DCM Chl fluor sum: 0.0
DCM Beam trans sum: 0.0
DCM O2 Mm/Kg sum: 0.0


In [9]:
# read from cnv with 'ignore' added to depSM
cast30 = ctd.from_cnv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/KM1919-St20-P2/St20_P2_cast30-align-cellTM-derive-bin.cnv")

# set up datetime to datenum 
ST_30 = pd.to_datetime(st30)

# split up and down casts; we want the downcast for the profile
down30, up30 = cast30.split()

# add a column with the start date and time in UTC (this is from the cnv header or from ship's log)
down30.insert(0, 'StartTime', pd.to_datetime(st30).replace(microsecond=0))

# slice the mixed layer out and save as csv
p2_30_ml = down30[down30['sigma-�00'].between(22.0,23.2)]
p2_30_ml.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/mixed_layer//Station20_cast30_ML.csv")

# get the sum of mixed layer slice oxygen, beam transmission, and chlorophyll fluor
cf_sum_p2_30_ml = p2_30_ml['flECO-AFL'].sum()
print('ML Chl fluor sum:', cf_sum_p2_30_ml)

bt_sum_p2_30_ml = p2_30_ml['CStarTr0'].sum()
print('ML Beam trans sum:', bt_sum_p2_30_ml)

o2_sum_p2_30_ml = p2_30_ml['sbeox0Mm/L'].sum()
print('ML O2 Mm/Kg sum:', o2_sum_p2_30_ml)

# Make lists of the input for the ML integrated dataframe
ML_data_30 = {'Cast':[30], 'StartTime':st30, 'StartTime matlab datenum': datetime_to_datenum(ST_30), \
           'Int chl fluor': cf_sum_p2_30_ml, \
           'Int beam trans': bt_sum_p2_30_ml, \
            'Int O2 Mm/Kg': o2_sum_p2_30_ml}

# slice the primary chlorophyll max out and save as csv
p2_30_pcm = down30[down30['sigma-�00'].between(23.2,30.7)]
p2_30_pcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/primary_chl/Station20_cast30_PCM.csv")

# get the sum of primary chl slice oxygen and beam transmission
cf_sum_p2_30_pcm = p2_30_pcm['flECO-AFL'].sum()
print('PCM Chl fluor sum:', cf_sum_p2_30_pcm)

bt_sum_p2_30_pcm = p2_30_pcm['CStarTr0'].sum()
print('PCM Beam trans sum:', bt_sum_p2_30_pcm)

o2_sum_p2_30_pcm = p2_30_pcm['sbeox0Mm/L'].sum()
print('PCM O2 Mm/Kg sum:', o2_sum_p2_30_pcm)

# Make lists of the input for the PCM integrated dataframe
PCM_data_30 = {'Cast':[30], 'StartTime':st30, 'StartTime matlab datenum': datetime_to_datenum(ST_30), \
            'Int chl fluor': cf_sum_p2_30_pcm, \
           'Int beam trans': bt_sum_p2_30_pcm, \
            'Int O2 Mm/Kg': o2_sum_p2_30_pcm}

# slice the deep chlorophyll max out and save as csv
p2_30_dcm = down30[down30['sigma-�00'].between(30.7,26.2)]
p2_30_dcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/deep_chl/Station20_cast30_DCM.csv")

# get the sum of deep chl slice oxygen and beam transmission
cf_sum_p2_30_dcm = p2_30_dcm['flECO-AFL'].sum()
print('DCM Chl fluor sum:', cf_sum_p2_30_dcm)

bt_sum_p2_30_dcm = p2_30_dcm['CStarTr0'].sum()
print('DCM Beam trans sum:', bt_sum_p2_30_dcm)

o2_sum_p2_30_dcm = p2_30_dcm['sbeox0Mm/L'].sum()
print('DCM O2 Mm/Kg sum:', o2_sum_p2_30_dcm)

# Make lists of the input for the DCM integrated dataframe
DCM_data_30 = {'Cast':[30], 'StartTime':st30, 'StartTime matlab datenum': datetime_to_datenum(ST_30), \
            'Int chl fluor': cf_sum_p2_30_dcm, \
           'Int beam trans': bt_sum_p2_30_dcm, \
            'Int O2 Mm/Kg': o2_sum_p2_30_dcm}

ML Chl fluor sum: 4.9495000000000005
ML Beam trans sum: 1570.6556
ML O2 Mm/Kg sum: 3497.1059999999998
PCM Chl fluor sum: 107.7809
PCM Beam trans sum: 96023.5814
PCM O2 Mm/Kg sum: 7054.908
DCM Chl fluor sum: 0.0
DCM Beam trans sum: 0.0
DCM O2 Mm/Kg sum: 0.0


In [10]:
# read from cnv with 'ignore' added to depSM
cast31 = ctd.from_cnv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/KM1919-St20-P2/St20_P2_cast31-align-cellTM-derive-bin.cnv")

# set up datetime to datenum 
ST_31 = pd.to_datetime(st31)

# split up and down casts; we want the downcast for the profile
down31, up31 = cast31.split()

# add a column with the start date and time in UTC (this is from the cnv header or from ship's log)
down31.insert(0, 'StartTime', pd.to_datetime(st31).replace(microsecond=0))

# slice the mixed layer out and save as csv
p2_31_ml = down31[down31['sigma-�00'].between(22.0,23.2)]
p2_31_ml.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/mixed_layer//Station20_cast31_ML.csv")

# get the sum of mixed layer slice oxygen, beam transmission, and chlorophyll fluor
cf_sum_p2_31_ml = p2_31_ml['flECO-AFL'].sum()
print('ML Chl fluor sum:', cf_sum_p2_31_ml)

bt_sum_p2_31_ml = p2_31_ml['CStarTr0'].sum()
print('ML Beam trans sum:', bt_sum_p2_31_ml)

o2_sum_p2_31_ml = p2_31_ml['sbeox0Mm/L'].sum()
print('ML O2 Mm/Kg sum:', o2_sum_p2_31_ml)

# Make lists of the input for the ML integrated dataframe
ML_data_31 = {'Cast':[31], 'StartTime':st31, 'StartTime matlab datenum': datetime_to_datenum(ST_31), \
           'Int chl fluor': cf_sum_p2_31_ml, \
           'Int beam trans': bt_sum_p2_31_ml, \
            'Int O2 Mm/Kg': o2_sum_p2_31_ml}

# slice the primary chlorophyll max out and save as csv
p2_31_pcm = down31[down31['sigma-�00'].between(23.2,31.7)]
p2_31_pcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/primary_chl/Station20_cast31_PCM.csv")

# get the sum of primary chl slice oxygen and beam transmission
cf_sum_p2_31_pcm = p2_31_pcm['flECO-AFL'].sum()
print('PCM Chl fluor sum:', cf_sum_p2_31_pcm)

bt_sum_p2_31_pcm = p2_31_pcm['CStarTr0'].sum()
print('PCM Beam trans sum:', bt_sum_p2_31_pcm)

o2_sum_p2_31_pcm = p2_31_pcm['sbeox0Mm/L'].sum()
print('PCM O2 Mm/Kg sum:', o2_sum_p2_31_pcm)

# Make lists of the input for the PCM integrated dataframe
PCM_data_31 = {'Cast':[31], 'StartTime':st31, 'StartTime matlab datenum': datetime_to_datenum(ST_31), \
            'Int chl fluor': cf_sum_p2_31_pcm, \
           'Int beam trans': bt_sum_p2_31_pcm, \
            'Int O2 Mm/Kg': o2_sum_p2_31_pcm}

# slice the deep chlorophyll max out and save as csv
p2_31_dcm = down31[down31['sigma-�00'].between(31.7,26.2)]
p2_31_dcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/deep_chl/Station20_cast31_DCM.csv")

# get the sum of deep chl slice oxygen and beam transmission
cf_sum_p2_31_dcm = p2_31_dcm['flECO-AFL'].sum()
print('DCM Chl fluor sum:', cf_sum_p2_31_dcm)

bt_sum_p2_31_dcm = p2_31_dcm['CStarTr0'].sum()
print('DCM Beam trans sum:', bt_sum_p2_31_dcm)

o2_sum_p2_31_dcm = p2_31_dcm['sbeox0Mm/L'].sum()
print('DCM O2 Mm/Kg sum:', o2_sum_p2_31_dcm)

# Make lists of the input for the DCM integrated dataframe
DCM_data_31 = {'Cast':[31], 'StartTime':st31, 'StartTime matlab datenum': datetime_to_datenum(ST_31), \
            'Int chl fluor': cf_sum_p2_31_dcm, \
           'Int beam trans': bt_sum_p2_31_dcm, \
            'Int O2 Mm/Kg': o2_sum_p2_31_dcm}

ML Chl fluor sum: 6.035299999999999
ML Beam trans sum: 1867.5778999999998
ML O2 Mm/Kg sum: 4127.896
PCM Chl fluor sum: 100.4304
PCM Beam trans sum: 94117.0924
PCM O2 Mm/Kg sum: 6224.446
DCM Chl fluor sum: 0.0
DCM Beam trans sum: 0.0
DCM O2 Mm/Kg sum: 0.0


In [11]:
# read from cnv with 'ignore' added to depSM
cast32 = ctd.from_cnv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/KM1919-St20-P2/St20_P2_cast32-align-cellTM-derive-bin.cnv")

# set up datetime to datenum 
ST_32 = pd.to_datetime(st32)

# split up and down casts; we want the downcast for the profile
down32, up32 = cast32.split()

# add a column with the start date and time in UTC (this is from the cnv header or from ship's log)
down32.insert(0, 'StartTime', pd.to_datetime(st32).replace(microsecond=0))

# slice the mixed layer out and save as csv
p2_32_ml = down32[down32['sigma-�00'].between(22.0,23.2)]
p2_32_ml.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/mixed_layer//Station20_cast32_ML.csv")

# get the sum of mixed layer slice oxygen, beam transmission, and chlorophyll fluor
cf_sum_p2_32_ml = p2_32_ml['flECO-AFL'].sum()
print('ML Chl fluor sum:', cf_sum_p2_32_ml)

bt_sum_p2_32_ml = p2_32_ml['CStarTr0'].sum()
print('ML Beam trans sum:', bt_sum_p2_32_ml)

o2_sum_p2_32_ml = p2_32_ml['sbeox0Mm/L'].sum()
print('ML O2 Mm/Kg sum:', o2_sum_p2_32_ml)

# Make lists of the input for the ML integrated dataframe
ML_data_32 = {'Cast':[32], 'StartTime':st32, 'StartTime matlab datenum': datetime_to_datenum(ST_32), \
           'Int chl fluor': cf_sum_p2_32_ml, \
           'Int beam trans': bt_sum_p2_32_ml, \
            'Int O2 Mm/Kg': o2_sum_p2_32_ml}

# slice the primary chlorophyll max out and save as csv
p2_32_pcm = down32[down32['sigma-�00'].between(23.2,32.7)]
p2_32_pcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/primary_chl/Station20_cast32_PCM.csv")

# get the sum of primary chl slice oxygen and beam transmission
cf_sum_p2_32_pcm = p2_32_pcm['flECO-AFL'].sum()
print('PCM Chl fluor sum:', cf_sum_p2_32_pcm)

bt_sum_p2_32_pcm = p2_32_pcm['CStarTr0'].sum()
print('PCM Beam trans sum:', bt_sum_p2_32_pcm)

o2_sum_p2_32_pcm = p2_32_pcm['sbeox0Mm/L'].sum()
print('PCM O2 Mm/Kg sum:', o2_sum_p2_32_pcm)

# Make lists of the input for the PCM integrated dataframe
PCM_data_32 = {'Cast':[32], 'StartTime':st32, 'StartTime matlab datenum': datetime_to_datenum(ST_32), \
            'Int chl fluor': cf_sum_p2_32_pcm, \
           'Int beam trans': bt_sum_p2_32_pcm, \
            'Int O2 Mm/Kg': o2_sum_p2_32_pcm}

# slice the deep chlorophyll max out and save as csv
p2_32_dcm = down32[down32['sigma-�00'].between(32.7,26.2)]
p2_32_dcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/deep_chl/Station20_cast32_DCM.csv")

# get the sum of deep chl slice oxygen and beam transmission
cf_sum_p2_32_dcm = p2_32_dcm['flECO-AFL'].sum()
print('DCM Chl fluor sum:', cf_sum_p2_32_dcm)

bt_sum_p2_32_dcm = p2_32_dcm['CStarTr0'].sum()
print('DCM Beam trans sum:', bt_sum_p2_32_dcm)

o2_sum_p2_32_dcm = p2_32_dcm['sbeox0Mm/L'].sum()
print('DCM O2 Mm/Kg sum:', o2_sum_p2_32_dcm)

# Make lists of the input for the DCM integrated dataframe
DCM_data_32 = {'Cast':[32], 'StartTime':st32, 'StartTime matlab datenum': datetime_to_datenum(ST_32), \
            'Int chl fluor': cf_sum_p2_32_dcm, \
           'Int beam trans': bt_sum_p2_32_dcm, \
            'Int O2 Mm/Kg': o2_sum_p2_32_dcm}

ML Chl fluor sum: 4.100700000000001
ML Beam trans sum: 1467.1161000000002
ML O2 Mm/Kg sum: 3278.642
PCM Chl fluor sum: 83.68520000000001
PCM Beam trans sum: 44836.602999999996
PCM O2 Mm/Kg sum: 6322.003000000001
DCM Chl fluor sum: 0.0
DCM Beam trans sum: 0.0
DCM O2 Mm/Kg sum: 0.0


In [12]:
# read from cnv with 'ignore' added to depSM
cast33 = ctd.from_cnv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/KM1919-St20-P2/St20_P2_cast33-align-cellTM-derive-bin.cnv")

# set up datetime to datenum 
ST_33 = pd.to_datetime(st33)

# split up and down casts; we want the downcast for the profile
down33, up33 = cast33.split()

# add a column with the start date and time in UTC (this is from the cnv header or from ship's log)
down33.insert(0, 'StartTime', pd.to_datetime(st33).replace(microsecond=0))

# slice the mixed layer out and save as csv
p2_33_ml = down33[down33['sigma-�00'].between(22.0,23.2)]
p2_33_ml.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/mixed_layer//Station20_cast33_ML.csv")

# get the sum of mixed layer slice oxygen, beam transmission, and chlorophyll fluor
cf_sum_p2_33_ml = p2_33_ml['flECO-AFL'].sum()
print('ML Chl fluor sum:', cf_sum_p2_33_ml)

bt_sum_p2_33_ml = p2_33_ml['CStarTr0'].sum()
print('ML Beam trans sum:', bt_sum_p2_33_ml)

o2_sum_p2_33_ml = p2_33_ml['sbeox0Mm/L'].sum()
print('ML O2 Mm/Kg sum:', o2_sum_p2_33_ml)

# Make lists of the input for the ML integrated dataframe
ML_data_33 = {'Cast':[33], 'StartTime':st33, 'StartTime matlab datenum': datetime_to_datenum(ST_33), \
           'Int chl fluor': cf_sum_p2_33_ml, \
           'Int beam trans': bt_sum_p2_33_ml, \
            'Int O2 Mm/Kg': o2_sum_p2_33_ml}

# slice the primary chlorophyll max out and save as csv
p2_33_pcm = down33[down33['sigma-�00'].between(23.2,33.7)]
p2_33_pcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/primary_chl/Station20_cast33_PCM.csv")

# get the sum of primary chl slice oxygen and beam transmission
cf_sum_p2_33_pcm = p2_33_pcm['flECO-AFL'].sum()
print('PCM Chl fluor sum:', cf_sum_p2_33_pcm)

bt_sum_p2_33_pcm = p2_33_pcm['CStarTr0'].sum()
print('PCM Beam trans sum:', bt_sum_p2_33_pcm)

o2_sum_p2_33_pcm = p2_33_pcm['sbeox0Mm/L'].sum()
print('PCM O2 Mm/Kg sum:', o2_sum_p2_33_pcm)

# Make lists of the input for the PCM integrated dataframe
PCM_data_33 = {'Cast':[33], 'StartTime':st33, 'StartTime matlab datenum': datetime_to_datenum(ST_33), \
            'Int chl fluor': cf_sum_p2_33_pcm, \
           'Int beam trans': bt_sum_p2_33_pcm, \
            'Int O2 Mm/Kg': o2_sum_p2_33_pcm}

# slice the deep chlorophyll max out and save as csv
p2_33_dcm = down33[down33['sigma-�00'].between(33.7,26.2)]
p2_33_dcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/deep_chl/Station20_cast33_DCM.csv")

# get the sum of deep chl slice oxygen and beam transmission
cf_sum_p2_33_dcm = p2_33_dcm['flECO-AFL'].sum()
print('DCM Chl fluor sum:', cf_sum_p2_33_dcm)

bt_sum_p2_33_dcm = p2_33_dcm['CStarTr0'].sum()
print('DCM Beam trans sum:', bt_sum_p2_33_dcm)

o2_sum_p2_33_dcm = p2_33_dcm['sbeox0Mm/L'].sum()
print('DCM O2 Mm/Kg sum:', o2_sum_p2_33_dcm)

# Make lists of the input for the DCM integrated dataframe
DCM_data_33 = {'Cast':[33], 'StartTime':st33, 'StartTime matlab datenum': datetime_to_datenum(ST_33), \
            'Int chl fluor': cf_sum_p2_33_dcm, \
           'Int beam trans': bt_sum_p2_33_dcm, \
            'Int O2 Mm/Kg': o2_sum_p2_33_dcm}

ML Chl fluor sum: 3.4269000000000003
ML Beam trans sum: 1277.9139
ML O2 Mm/Kg sum: 2817.3869999999997
PCM Chl fluor sum: 61.296699999999994
PCM Beam trans sum: 15347.481600000003
PCM O2 Mm/Kg sum: 7461.6359999999995
DCM Chl fluor sum: 0.0
DCM Beam trans sum: 0.0
DCM O2 Mm/Kg sum: 0.0


In [13]:
# read from cnv with 'ignore' added to depSM
cast34 = ctd.from_cnv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/KM1919-St20-P2/St20_P2_cast34-align-cellTM-derive-bin.cnv")

# set up datetime to datenum 
ST_34 = pd.to_datetime(st34)

# split up and down casts; we want the downcast for the profile
down34, up34 = cast34.split()

# add a column with the start date and time in UTC (this is from the cnv header or from ship's log)
down34.insert(0, 'StartTime', pd.to_datetime(st34).replace(microsecond=0))

# slice the mixed layer out and save as csv
p2_34_ml = down34[down34['sigma-�00'].between(22.0,23.2)]
p2_34_ml.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/mixed_layer//Station20_cast34_ML.csv")

# get the sum of mixed layer slice oxygen, beam transmission, and chlorophyll fluor
cf_sum_p2_34_ml = p2_34_ml['flECO-AFL'].sum()
print('ML Chl fluor sum:', cf_sum_p2_34_ml)

bt_sum_p2_34_ml = p2_34_ml['CStarTr0'].sum()
print('ML Beam trans sum:', bt_sum_p2_34_ml)

o2_sum_p2_34_ml = p2_34_ml['sbeox0Mm/L'].sum()
print('ML O2 Mm/Kg sum:', o2_sum_p2_34_ml)

# Make lists of the input for the ML integrated dataframe
ML_data_34 = {'Cast':[34], 'StartTime':st34, 'StartTime matlab datenum': datetime_to_datenum(ST_34), \
           'Int chl fluor': cf_sum_p2_34_ml, \
           'Int beam trans': bt_sum_p2_34_ml, \
            'Int O2 Mm/Kg': o2_sum_p2_34_ml}

# slice the primary chlorophyll max out and save as csv
p2_34_pcm = down34[down34['sigma-�00'].between(23.2,34.7)]
p2_34_pcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/primary_chl/Station20_cast34_PCM.csv")

# get the sum of primary chl slice oxygen and beam transmission
cf_sum_p2_34_pcm = p2_34_pcm['flECO-AFL'].sum()
print('PCM Chl fluor sum:', cf_sum_p2_34_pcm)

bt_sum_p2_34_pcm = p2_34_pcm['CStarTr0'].sum()
print('PCM Beam trans sum:', bt_sum_p2_34_pcm)

o2_sum_p2_34_pcm = p2_34_pcm['sbeox0Mm/L'].sum()
print('PCM O2 Mm/Kg sum:', o2_sum_p2_34_pcm)

# Make lists of the input for the PCM integrated dataframe
PCM_data_34 = {'Cast':[34], 'StartTime':st34, 'StartTime matlab datenum': datetime_to_datenum(ST_34), \
            'Int chl fluor': cf_sum_p2_34_pcm, \
           'Int beam trans': bt_sum_p2_34_pcm, \
            'Int O2 Mm/Kg': o2_sum_p2_34_pcm}

# slice the deep chlorophyll max out and save as csv
p2_34_dcm = down34[down34['sigma-�00'].between(34.7,26.2)]
p2_34_dcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/deep_chl/Station20_cast34_DCM.csv")

# get the sum of deep chl slice oxygen and beam transmission
cf_sum_p2_34_dcm = p2_34_dcm['flECO-AFL'].sum()
print('DCM Chl fluor sum:', cf_sum_p2_34_dcm)

bt_sum_p2_34_dcm = p2_34_dcm['CStarTr0'].sum()
print('DCM Beam trans sum:', bt_sum_p2_34_dcm)

o2_sum_p2_34_dcm = p2_34_dcm['sbeox0Mm/L'].sum()
print('DCM O2 Mm/Kg sum:', o2_sum_p2_34_dcm)

# Make lists of the input for the DCM integrated dataframe
DCM_data_34 = {'Cast':[34], 'StartTime':st34, 'StartTime matlab datenum': datetime_to_datenum(ST_34), \
            'Int chl fluor': cf_sum_p2_34_dcm, \
           'Int beam trans': bt_sum_p2_34_dcm, \
            'Int O2 Mm/Kg': o2_sum_p2_34_dcm}

ML Chl fluor sum: 1.9979
ML Beam trans sum: 879.6099999999999
ML O2 Mm/Kg sum: 1945.229
PCM Chl fluor sum: 89.38369999999999
PCM Beam trans sum: 65347.5107
PCM O2 Mm/Kg sum: 7943.815999999999
DCM Chl fluor sum: 0.0
DCM Beam trans sum: 0.0
DCM O2 Mm/Kg sum: 0.0


In [14]:
# read from cnv with 'ignore' added to depSM
cast35 = ctd.from_cnv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/KM1919-St20-P2/St20_P2_cast35-align-cellTM-derive-bin.cnv")

# set up datetime to datenum 
ST_35 = pd.to_datetime(st35)

# split up and down casts; we want the downcast for the profile
down35, up35 = cast35.split()

# add a column with the start date and time in UTC (this is from the cnv header or from ship's log)
down35.insert(0, 'StartTime', pd.to_datetime(st35).replace(microsecond=0))

# slice the mixed layer out and save as csv
p2_35_ml = down35[down35['sigma-�00'].between(22.0,23.2)]
p2_35_ml.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/mixed_layer//Station20_cast35_ML.csv")

# get the sum of mixed layer slice oxygen, beam transmission, and chlorophyll fluor
cf_sum_p2_35_ml = p2_35_ml['flECO-AFL'].sum()
print('ML Chl fluor sum:', cf_sum_p2_35_ml)

bt_sum_p2_35_ml = p2_35_ml['CStarTr0'].sum()
print('ML Beam trans sum:', bt_sum_p2_35_ml)

o2_sum_p2_35_ml = p2_35_ml['sbeox0Mm/L'].sum()
print('ML O2 Mm/Kg sum:', o2_sum_p2_35_ml)

# Make lists of the input for the ML integrated dataframe
ML_data_35 = {'Cast':[35], 'StartTime':st35, 'StartTime matlab datenum': datetime_to_datenum(ST_35), \
           'Int chl fluor': cf_sum_p2_35_ml, \
           'Int beam trans': bt_sum_p2_35_ml, \
            'Int O2 Mm/Kg': o2_sum_p2_35_ml}

# slice the primary chlorophyll max out and save as csv
p2_35_pcm = down35[down35['sigma-�00'].between(23.2,35.7)]
p2_35_pcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/primary_chl/Station20_cast35_PCM.csv")

# get the sum of primary chl slice oxygen and beam transmission
cf_sum_p2_35_pcm = p2_35_pcm['flECO-AFL'].sum()
print('PCM Chl fluor sum:', cf_sum_p2_35_pcm)

bt_sum_p2_35_pcm = p2_35_pcm['CStarTr0'].sum()
print('PCM Beam trans sum:', bt_sum_p2_35_pcm)

o2_sum_p2_35_pcm = p2_35_pcm['sbeox0Mm/L'].sum()
print('PCM O2 Mm/Kg sum:', o2_sum_p2_35_pcm)

# Make lists of the input for the PCM integrated dataframe
PCM_data_35 = {'Cast':[35], 'StartTime':st35, 'StartTime matlab datenum': datetime_to_datenum(ST_35), \
            'Int chl fluor': cf_sum_p2_35_pcm, \
           'Int beam trans': bt_sum_p2_35_pcm, \
            'Int O2 Mm/Kg': o2_sum_p2_35_pcm}

# slice the deep chlorophyll max out and save as csv
p2_35_dcm = down35[down35['sigma-�00'].between(35.7,26.2)]
p2_35_dcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/deep_chl/Station20_cast35_DCM.csv")

# get the sum of deep chl slice oxygen and beam transmission
cf_sum_p2_35_dcm = p2_35_dcm['flECO-AFL'].sum()
print('DCM Chl fluor sum:', cf_sum_p2_35_dcm)

bt_sum_p2_35_dcm = p2_35_dcm['CStarTr0'].sum()
print('DCM Beam trans sum:', bt_sum_p2_35_dcm)

o2_sum_p2_35_dcm = p2_35_dcm['sbeox0Mm/L'].sum()
print('DCM O2 Mm/Kg sum:', o2_sum_p2_35_dcm)

# Make lists of the input for the DCM integrated dataframe
DCM_data_35 = {'Cast':[35], 'StartTime':st35, 'StartTime matlab datenum': datetime_to_datenum(ST_35), \
            'Int chl fluor': cf_sum_p2_35_dcm, \
           'Int beam trans': bt_sum_p2_35_dcm, \
            'Int O2 Mm/Kg': o2_sum_p2_35_dcm}

ML Chl fluor sum: 3.8921
ML Beam trans sum: 1366.4841
ML O2 Mm/Kg sum: 3034.7050000000004
PCM Chl fluor sum: 118.51740000000001
PCM Beam trans sum: 114689.1344
PCM O2 Mm/Kg sum: 11533.641
DCM Chl fluor sum: 0.0
DCM Beam trans sum: 0.0
DCM O2 Mm/Kg sum: 0.0


In [15]:
# read from cnv with 'ignore' added to depSM
cast36 = ctd.from_cnv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/KM1919-St20-P2/St20_P2_cast36-align-cellTM-derive-bin.cnv")

# set up datetime to datenum 
ST_36 = pd.to_datetime(st36)

# split up and down casts; we want the downcast for the profile
down36, up36 = cast36.split()

# add a column with the start date and time in UTC (this is from the cnv header or from ship's log)
down36.insert(0, 'StartTime', pd.to_datetime(st36).replace(microsecond=0))

# slice the mixed layer out and save as csv
p2_36_ml = down36[down36['sigma-�00'].between(22.0,23.2)]
p2_36_ml.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/mixed_layer//Station20_cast36_ML.csv")

# get the sum of mixed layer slice oxygen, beam transmission, and chlorophyll fluor
cf_sum_p2_36_ml = p2_36_ml['flECO-AFL'].sum()
print('ML Chl fluor sum:', cf_sum_p2_36_ml)

bt_sum_p2_36_ml = p2_36_ml['CStarTr0'].sum()
print('ML Beam trans sum:', bt_sum_p2_36_ml)

o2_sum_p2_36_ml = p2_36_ml['sbeox0Mm/L'].sum()
print('ML O2 Mm/Kg sum:', o2_sum_p2_36_ml)

# Make lists of the input for the ML integrated dataframe
ML_data_36 = {'Cast':[36], 'StartTime':st36, 'StartTime matlab datenum': datetime_to_datenum(ST_36), \
           'Int chl fluor': cf_sum_p2_36_ml, \
           'Int beam trans': bt_sum_p2_36_ml, \
            'Int O2 Mm/Kg': o2_sum_p2_36_ml}

# slice the primary chlorophyll max out and save as csv
p2_36_pcm = down36[down36['sigma-�00'].between(23.2,36.7)]
p2_36_pcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/primary_chl/Station20_cast36_PCM.csv")

# get the sum of primary chl slice oxygen and beam transmission
cf_sum_p2_36_pcm = p2_36_pcm['flECO-AFL'].sum()
print('PCM Chl fluor sum:', cf_sum_p2_36_pcm)

bt_sum_p2_36_pcm = p2_36_pcm['CStarTr0'].sum()
print('PCM Beam trans sum:', bt_sum_p2_36_pcm)

o2_sum_p2_36_pcm = p2_36_pcm['sbeox0Mm/L'].sum()
print('PCM O2 Mm/Kg sum:', o2_sum_p2_36_pcm)

# Make lists of the input for the PCM integrated dataframe
PCM_data_36 = {'Cast':[36], 'StartTime':st36, 'StartTime matlab datenum': datetime_to_datenum(ST_36), \
            'Int chl fluor': cf_sum_p2_36_pcm, \
           'Int beam trans': bt_sum_p2_36_pcm, \
            'Int O2 Mm/Kg': o2_sum_p2_36_pcm}

# slice the deep chlorophyll max out and save as csv
p2_36_dcm = down36[down36['sigma-�00'].between(36.7,26.2)]
p2_36_dcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/deep_chl/Station20_cast36_DCM.csv")

# get the sum of deep chl slice oxygen and beam transmission
cf_sum_p2_36_dcm = p2_36_dcm['flECO-AFL'].sum()
print('DCM Chl fluor sum:', cf_sum_p2_36_dcm)

bt_sum_p2_36_dcm = p2_36_dcm['CStarTr0'].sum()
print('DCM Beam trans sum:', bt_sum_p2_36_dcm)

o2_sum_p2_36_dcm = p2_36_dcm['sbeox0Mm/L'].sum()
print('DCM O2 Mm/Kg sum:', o2_sum_p2_36_dcm)

# Make lists of the input for the DCM integrated dataframe
DCM_data_36 = {'Cast':[36], 'StartTime':st36, 'StartTime matlab datenum': datetime_to_datenum(ST_36), \
            'Int chl fluor': cf_sum_p2_36_dcm, \
           'Int beam trans': bt_sum_p2_36_dcm, \
            'Int O2 Mm/Kg': o2_sum_p2_36_dcm}

ML Chl fluor sum: 4.227200000000001
ML Beam trans sum: 1366.7175
ML O2 Mm/Kg sum: 3070.006
PCM Chl fluor sum: 125.38629999999999
PCM Beam trans sum: 119485.4325
PCM O2 Mm/Kg sum: 12619.73
DCM Chl fluor sum: 0.0
DCM Beam trans sum: 0.0
DCM O2 Mm/Kg sum: 0.0


In [16]:
# read from cnv with 'ignore' added to depSM
cast37 = ctd.from_cnv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/KM1919-St20-P2/St20_P2_cast37-align-cellTM-derive-bin.cnv")

# set up datetime to datenum 
ST_37 = pd.to_datetime(st37)

# split up and down casts; we want the downcast for the profile
down37, up37 = cast37.split()

# add a column with the start date and time in UTC (this is from the cnv header or from ship's log)
down37.insert(0, 'StartTime', pd.to_datetime(st37).replace(microsecond=0))

# slice the mixed layer out and save as csv
p2_37_ml = down37[down37['sigma-�00'].between(22.0,23.2)]
p2_37_ml.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/mixed_layer//Station20_cast37_ML.csv")

# get the sum of mixed layer slice oxygen, beam transmission, and chlorophyll fluor
cf_sum_p2_37_ml = p2_37_ml['flECO-AFL'].sum()
print('ML Chl fluor sum:', cf_sum_p2_37_ml)

bt_sum_p2_37_ml = p2_37_ml['CStarTr0'].sum()
print('ML Beam trans sum:', bt_sum_p2_37_ml)

o2_sum_p2_37_ml = p2_37_ml['sbeox0Mm/L'].sum()
print('ML O2 Mm/Kg sum:', o2_sum_p2_37_ml)

# Make lists of the input for the ML integrated dataframe
ML_data_37 = {'Cast':[37], 'StartTime':st37, 'StartTime matlab datenum': datetime_to_datenum(ST_37), \
           'Int chl fluor': cf_sum_p2_37_ml, \
           'Int beam trans': bt_sum_p2_37_ml, \
            'Int O2 Mm/Kg': o2_sum_p2_37_ml}

# slice the primary chlorophyll max out and save as csv
p2_37_pcm = down37[down37['sigma-�00'].between(23.2,37.7)]
p2_37_pcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/primary_chl/Station20_cast37_PCM.csv")

# get the sum of primary chl slice oxygen and beam transmission
cf_sum_p2_37_pcm = p2_37_pcm['flECO-AFL'].sum()
print('PCM Chl fluor sum:', cf_sum_p2_37_pcm)

bt_sum_p2_37_pcm = p2_37_pcm['CStarTr0'].sum()
print('PCM Beam trans sum:', bt_sum_p2_37_pcm)

o2_sum_p2_37_pcm = p2_37_pcm['sbeox0Mm/L'].sum()
print('PCM O2 Mm/Kg sum:', o2_sum_p2_37_pcm)

# Make lists of the input for the PCM integrated dataframe
PCM_data_37 = {'Cast':[37], 'StartTime':st37, 'StartTime matlab datenum': datetime_to_datenum(ST_37), \
            'Int chl fluor': cf_sum_p2_37_pcm, \
           'Int beam trans': bt_sum_p2_37_pcm, \
            'Int O2 Mm/Kg': o2_sum_p2_37_pcm}

# slice the deep chlorophyll max out and save as csv
p2_37_dcm = down37[down37['sigma-�00'].between(37.7,26.2)]
p2_37_dcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/deep_chl/Station20_cast37_DCM.csv")

# get the sum of deep chl slice oxygen and beam transmission
cf_sum_p2_37_dcm = p2_37_dcm['flECO-AFL'].sum()
print('DCM Chl fluor sum:', cf_sum_p2_37_dcm)

bt_sum_p2_37_dcm = p2_37_dcm['CStarTr0'].sum()
print('DCM Beam trans sum:', bt_sum_p2_37_dcm)

o2_sum_p2_37_dcm = p2_37_dcm['sbeox0Mm/L'].sum()
print('DCM O2 Mm/Kg sum:', o2_sum_p2_37_dcm)

# Make lists of the input for the DCM integrated dataframe
DCM_data_37 = {'Cast':[37], 'StartTime':st37, 'StartTime matlab datenum': datetime_to_datenum(ST_37), \
            'Int chl fluor': cf_sum_p2_37_dcm, \
           'Int beam trans': bt_sum_p2_37_dcm, \
            'Int O2 Mm/Kg': o2_sum_p2_37_dcm}

ML Chl fluor sum: 2.0193
ML Beam trans sum: 882.7343999999999
ML O2 Mm/Kg sum: 1967.009
PCM Chl fluor sum: 75.1198
PCM Beam trans sum: 45506.253899999996
PCM O2 Mm/Kg sum: 8046.352
DCM Chl fluor sum: 0.0
DCM Beam trans sum: 0.0
DCM O2 Mm/Kg sum: 0.0


In [17]:
# read from cnv with 'ignore' added to depSM
cast38 = ctd.from_cnv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/KM1919-St20-P2/St20_P2_cast38-align-cellTM-derive-bin.cnv")

# set up datetime to datenum 
ST_38 = pd.to_datetime(st38)

# split up and down casts; we want the downcast for the profile
down38, up38 = cast38.split()

# add a column with the start date and time in UTC (this is from the cnv header or from ship's log)
down38.insert(0, 'StartTime', pd.to_datetime(st38).replace(microsecond=0))

# slice the mixed layer out and save as csv
p2_38_ml = down38[down38['sigma-�00'].between(22.0,23.2)]
p2_38_ml.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/mixed_layer//Station20_cast38_ML.csv")

# get the sum of mixed layer slice oxygen, beam transmission, and chlorophyll fluor
cf_sum_p2_38_ml = p2_38_ml['flECO-AFL'].sum()
print('ML Chl fluor sum:', cf_sum_p2_38_ml)

bt_sum_p2_38_ml = p2_38_ml['CStarTr0'].sum()
print('ML Beam trans sum:', bt_sum_p2_38_ml)

o2_sum_p2_38_ml = p2_38_ml['sbeox0Mm/L'].sum()
print('ML O2 Mm/Kg sum:', o2_sum_p2_38_ml)

# Make lists of the input for the ML integrated dataframe
ML_data_38 = {'Cast':[38], 'StartTime':st38, 'StartTime matlab datenum': datetime_to_datenum(ST_38), \
           'Int chl fluor': cf_sum_p2_38_ml, \
           'Int beam trans': bt_sum_p2_38_ml, \
            'Int O2 Mm/Kg': o2_sum_p2_38_ml}

# slice the primary chlorophyll max out and save as csv
p2_38_pcm = down38[down38['sigma-�00'].between(23.2,38.7)]
p2_38_pcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/primary_chl/Station20_cast38_PCM.csv")

# get the sum of primary chl slice oxygen and beam transmission
cf_sum_p2_38_pcm = p2_38_pcm['flECO-AFL'].sum()
print('PCM Chl fluor sum:', cf_sum_p2_38_pcm)

bt_sum_p2_38_pcm = p2_38_pcm['CStarTr0'].sum()
print('PCM Beam trans sum:', bt_sum_p2_38_pcm)

o2_sum_p2_38_pcm = p2_38_pcm['sbeox0Mm/L'].sum()
print('PCM O2 Mm/Kg sum:', o2_sum_p2_38_pcm)

# Make lists of the input for the PCM integrated dataframe
PCM_data_38 = {'Cast':[38], 'StartTime':st38, 'StartTime matlab datenum': datetime_to_datenum(ST_38), \
            'Int chl fluor': cf_sum_p2_38_pcm, \
           'Int beam trans': bt_sum_p2_38_pcm, \
            'Int O2 Mm/Kg': o2_sum_p2_38_pcm}

# slice the deep chlorophyll max out and save as csv
p2_38_dcm = down38[down38['sigma-�00'].between(38.7,26.2)]
p2_38_dcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/deep_chl/Station20_cast38_DCM.csv")

# get the sum of deep chl slice oxygen and beam transmission
cf_sum_p2_38_dcm = p2_38_dcm['flECO-AFL'].sum()
print('DCM Chl fluor sum:', cf_sum_p2_38_dcm)

bt_sum_p2_38_dcm = p2_38_dcm['CStarTr0'].sum()
print('DCM Beam trans sum:', bt_sum_p2_38_dcm)

o2_sum_p2_38_dcm = p2_38_dcm['sbeox0Mm/L'].sum()
print('DCM O2 Mm/Kg sum:', o2_sum_p2_38_dcm)

# Make lists of the input for the DCM integrated dataframe
DCM_data_38 = {'Cast':[38], 'StartTime':st38, 'StartTime matlab datenum': datetime_to_datenum(ST_38), \
            'Int chl fluor': cf_sum_p2_38_dcm, \
           'Int beam trans': bt_sum_p2_38_dcm, \
            'Int O2 Mm/Kg': o2_sum_p2_38_dcm}

ML Chl fluor sum: 2.7379000000000002
ML Beam trans sum: 1078.2688
ML O2 Mm/Kg sum: 2426.6220000000003
PCM Chl fluor sum: 61.95779999999999
PCM Beam trans sum: 20280.4516
PCM O2 Mm/Kg sum: 8802.399
DCM Chl fluor sum: 0.0
DCM Beam trans sum: 0.0
DCM O2 Mm/Kg sum: 0.0


In [18]:
# read from cnv with 'ignore' added to depSM
cast39 = ctd.from_cnv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/KM1919-St20-P2/St20_P2_cast39-align-cellTM-derive-bin.cnv")

# set up datetime to datenum 
ST_39 = pd.to_datetime(st39)

# split up and down casts; we want the downcast for the profile
down39, up39 = cast39.split()

# add a column with the start date and time in UTC (this is from the cnv header or from ship's log)
down39.insert(0, 'StartTime', pd.to_datetime(st39).replace(microsecond=0))

# slice the mixed layer out and save as csv
p2_39_ml = down39[down39['sigma-�00'].between(22.0,23.2)]
p2_39_ml.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/mixed_layer//Station20_cast39_ML.csv")

# get the sum of mixed layer slice oxygen, beam transmission, and chlorophyll fluor
cf_sum_p2_39_ml = p2_39_ml['flECO-AFL'].sum()
print('ML Chl fluor sum:', cf_sum_p2_39_ml)

bt_sum_p2_39_ml = p2_39_ml['CStarTr0'].sum()
print('ML Beam trans sum:', bt_sum_p2_39_ml)

o2_sum_p2_39_ml = p2_39_ml['sbeox0Mm/L'].sum()
print('ML O2 Mm/Kg sum:', o2_sum_p2_39_ml)

# Make lists of the input for the ML integrated dataframe
ML_data_39 = {'Cast':[39], 'StartTime':st39, 'StartTime matlab datenum': datetime_to_datenum(ST_39), \
           'Int chl fluor': cf_sum_p2_39_ml, \
           'Int beam trans': bt_sum_p2_39_ml, \
            'Int O2 Mm/Kg': o2_sum_p2_39_ml}

# slice the primary chlorophyll max out and save as csv
p2_39_pcm = down39[down39['sigma-�00'].between(23.2,39.7)]
p2_39_pcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/primary_chl/Station20_cast39_PCM.csv")

# get the sum of primary chl slice oxygen and beam transmission
cf_sum_p2_39_pcm = p2_39_pcm['flECO-AFL'].sum()
print('PCM Chl fluor sum:', cf_sum_p2_39_pcm)

bt_sum_p2_39_pcm = p2_39_pcm['CStarTr0'].sum()
print('PCM Beam trans sum:', bt_sum_p2_39_pcm)

o2_sum_p2_39_pcm = p2_39_pcm['sbeox0Mm/L'].sum()
print('PCM O2 Mm/Kg sum:', o2_sum_p2_39_pcm)

# Make lists of the input for the PCM integrated dataframe
PCM_data_39 = {'Cast':[39], 'StartTime':st39, 'StartTime matlab datenum': datetime_to_datenum(ST_39), \
            'Int chl fluor': cf_sum_p2_39_pcm, \
           'Int beam trans': bt_sum_p2_39_pcm, \
            'Int O2 Mm/Kg': o2_sum_p2_39_pcm}

# slice the deep chlorophyll max out and save as csv
p2_39_dcm = down39[down39['sigma-�00'].between(39.7,26.2)]
p2_39_dcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/deep_chl/Station20_cast39_DCM.csv")

# get the sum of deep chl slice oxygen and beam transmission
cf_sum_p2_39_dcm = p2_39_dcm['flECO-AFL'].sum()
print('DCM Chl fluor sum:', cf_sum_p2_39_dcm)

bt_sum_p2_39_dcm = p2_39_dcm['CStarTr0'].sum()
print('DCM Beam trans sum:', bt_sum_p2_39_dcm)

o2_sum_p2_39_dcm = p2_39_dcm['sbeox0Mm/L'].sum()
print('DCM O2 Mm/Kg sum:', o2_sum_p2_39_dcm)

# Make lists of the input for the DCM integrated dataframe
DCM_data_39 = {'Cast':[39], 'StartTime':st39, 'StartTime matlab datenum': datetime_to_datenum(ST_39), \
            'Int chl fluor': cf_sum_p2_39_dcm, \
           'Int beam trans': bt_sum_p2_39_dcm, \
            'Int O2 Mm/Kg': o2_sum_p2_39_dcm}

ML Chl fluor sum: 4.216799999999999
ML Beam trans sum: 1658.1952
ML O2 Mm/Kg sum: 3765.645
PCM Chl fluor sum: 78.9871
PCM Beam trans sum: 44872.279599999994
PCM O2 Mm/Kg sum: 8238.554999999998
DCM Chl fluor sum: 0.0
DCM Beam trans sum: 0.0
DCM O2 Mm/Kg sum: 0.0


In [19]:
# read from cnv with 'ignore' added to depSM
cast40 = ctd.from_cnv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/KM1919-St20-P2/St20_P2_cast40-align-cellTM-derive-bin.cnv")

# set up datetime to datenum 
ST_40 = pd.to_datetime(st40)

# split up and down casts; we want the downcast for the profile
down40, up40 = cast40.split()

# add a column with the start date and time in UTC (this is from the cnv header or from ship's log)
down40.insert(0, 'StartTime', pd.to_datetime(st40).replace(microsecond=0))

# slice the mixed layer out and save as csv
p2_40_ml = down40[down40['sigma-�00'].between(22.0,23.2)]
p2_40_ml.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/mixed_layer//Station20_cast40_ML.csv")

# get the sum of mixed layer slice oxygen, beam transmission, and chlorophyll fluor
cf_sum_p2_40_ml = p2_40_ml['flECO-AFL'].sum()
print('ML Chl fluor sum:', cf_sum_p2_40_ml)

bt_sum_p2_40_ml = p2_40_ml['CStarTr0'].sum()
print('ML Beam trans sum:', bt_sum_p2_40_ml)

o2_sum_p2_40_ml = p2_40_ml['sbeox0Mm/L'].sum()
print('ML O2 Mm/Kg sum:', o2_sum_p2_40_ml)

# Make lists of the input for the ML integrated dataframe
ML_data_40 = {'Cast':[40], 'StartTime':st40, 'StartTime matlab datenum': datetime_to_datenum(ST_40), \
           'Int chl fluor': cf_sum_p2_40_ml, \
           'Int beam trans': bt_sum_p2_40_ml, \
            'Int O2 Mm/Kg': o2_sum_p2_40_ml}

# slice the primary chlorophyll max out and save as csv
p2_40_pcm = down40[down40['sigma-�00'].between(23.2,40.7)]
p2_40_pcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/primary_chl/Station20_cast40_PCM.csv")

# get the sum of primary chl slice oxygen and beam transmission
cf_sum_p2_40_pcm = p2_40_pcm['flECO-AFL'].sum()
print('PCM Chl fluor sum:', cf_sum_p2_40_pcm)

bt_sum_p2_40_pcm = p2_40_pcm['CStarTr0'].sum()
print('PCM Beam trans sum:', bt_sum_p2_40_pcm)

o2_sum_p2_40_pcm = p2_40_pcm['sbeox0Mm/L'].sum()
print('PCM O2 Mm/Kg sum:', o2_sum_p2_40_pcm)

# Make lists of the input for the PCM integrated dataframe
PCM_data_40 = {'Cast':[40], 'StartTime':st40, 'StartTime matlab datenum': datetime_to_datenum(ST_40), \
            'Int chl fluor': cf_sum_p2_40_pcm, \
           'Int beam trans': bt_sum_p2_40_pcm, \
            'Int O2 Mm/Kg': o2_sum_p2_40_pcm}

# slice the deep chlorophyll max out and save as csv
p2_40_dcm = down40[down40['sigma-�00'].between(40.7,26.2)]
p2_40_dcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/deep_chl/Station20_cast40_DCM.csv")

# get the sum of deep chl slice oxygen and beam transmission
cf_sum_p2_40_dcm = p2_40_dcm['flECO-AFL'].sum()
print('DCM Chl fluor sum:', cf_sum_p2_40_dcm)

bt_sum_p2_40_dcm = p2_40_dcm['CStarTr0'].sum()
print('DCM Beam trans sum:', bt_sum_p2_40_dcm)

o2_sum_p2_40_dcm = p2_40_dcm['sbeox0Mm/L'].sum()
print('DCM O2 Mm/Kg sum:', o2_sum_p2_40_dcm)

# Make lists of the input for the DCM integrated dataframe
DCM_data_40 = {'Cast':[40], 'StartTime':st40, 'StartTime matlab datenum': datetime_to_datenum(ST_40), \
            'Int chl fluor': cf_sum_p2_40_dcm, \
           'Int beam trans': bt_sum_p2_40_dcm, \
            'Int O2 Mm/Kg': o2_sum_p2_40_dcm}

ML Chl fluor sum: 3.2897
ML Beam trans sum: 1070.1795
ML O2 Mm/Kg sum: 2445.58
PCM Chl fluor sum: 57.499300000000005
PCM Beam trans sum: 15260.54
PCM O2 Mm/Kg sum: 8909.025
DCM Chl fluor sum: 0.0
DCM Beam trans sum: 0.0
DCM O2 Mm/Kg sum: 0.0


In [20]:
# read from cnv with 'ignore' added to depSM
cast42 = ctd.from_cnv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/KM1919-St20-P2/St20_P2_cast42-align-cellTM-derive-bin.cnv")

# set up datetime to datenum 
ST_42 = pd.to_datetime(st42)

# split up and down casts; we want the downcast for the profile
down42, up42 = cast42.split()

# add a column with the start date and time in UTC (this is from the cnv header or from ship's log)
down42.insert(0, 'StartTime', pd.to_datetime(st42).replace(microsecond=0))

# slice the mixed layer out and save as csv
p2_42_ml = down42[down42['sigma-�00'].between(22.0,23.2)]
p2_42_ml.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/mixed_layer//Station20_cast42_ML.csv")

# get the sum of mixed layer slice oxygen, beam transmission, and chlorophyll fluor
cf_sum_p2_42_ml = p2_42_ml['flECO-AFL'].sum()
print('ML Chl fluor sum:', cf_sum_p2_42_ml)

bt_sum_p2_42_ml = p2_42_ml['CStarTr0'].sum()
print('ML Beam trans sum:', bt_sum_p2_42_ml)

o2_sum_p2_42_ml = p2_42_ml['sbeox0Mm/L'].sum()
print('ML O2 Mm/Kg sum:', o2_sum_p2_42_ml)

# Make lists of the input for the ML integrated dataframe
ML_data_42 = {'Cast':[42], 'StartTime':st42, 'StartTime matlab datenum': datetime_to_datenum(ST_42), \
           'Int chl fluor': cf_sum_p2_42_ml, \
           'Int beam trans': bt_sum_p2_42_ml, \
            'Int O2 Mm/Kg': o2_sum_p2_42_ml}

# slice the primary chlorophyll max out and save as csv
p2_42_pcm = down42[down42['sigma-�00'].between(23.2,42.7)]
p2_42_pcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/primary_chl/Station20_cast42_PCM.csv")

# get the sum of primary chl slice oxygen and beam transmission
cf_sum_p2_42_pcm = p2_42_pcm['flECO-AFL'].sum()
print('PCM Chl fluor sum:', cf_sum_p2_42_pcm)

bt_sum_p2_42_pcm = p2_42_pcm['CStarTr0'].sum()
print('PCM Beam trans sum:', bt_sum_p2_42_pcm)

o2_sum_p2_42_pcm = p2_42_pcm['sbeox0Mm/L'].sum()
print('PCM O2 Mm/Kg sum:', o2_sum_p2_42_pcm)

# Make lists of the input for the PCM integrated dataframe
PCM_data_42 = {'Cast':[42], 'StartTime':st42, 'StartTime matlab datenum': datetime_to_datenum(ST_42), \
            'Int chl fluor': cf_sum_p2_42_pcm, \
           'Int beam trans': bt_sum_p2_42_pcm, \
            'Int O2 Mm/Kg': o2_sum_p2_42_pcm}

# slice the deep chlorophyll max out and save as csv
p2_42_dcm = down42[down42['sigma-�00'].between(42.7,26.2)]
p2_42_dcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/deep_chl/Station20_cast42_DCM.csv")

# get the sum of deep chl slice oxygen and beam transmission
cf_sum_p2_42_dcm = p2_42_dcm['flECO-AFL'].sum()
print('DCM Chl fluor sum:', cf_sum_p2_42_dcm)

bt_sum_p2_42_dcm = p2_42_dcm['CStarTr0'].sum()
print('DCM Beam trans sum:', bt_sum_p2_42_dcm)

o2_sum_p2_42_dcm = p2_42_dcm['sbeox0Mm/L'].sum()
print('DCM O2 Mm/Kg sum:', o2_sum_p2_42_dcm)

# Make lists of the input for the DCM integrated dataframe
DCM_data_42 = {'Cast':[42], 'StartTime':st42, 'StartTime matlab datenum': datetime_to_datenum(ST_42), \
            'Int chl fluor': cf_sum_p2_42_dcm, \
           'Int beam trans': bt_sum_p2_42_dcm, \
            'Int O2 Mm/Kg': o2_sum_p2_42_dcm}

ML Chl fluor sum: 2.9873000000000003
ML Beam trans sum: 1173.662
ML O2 Mm/Kg sum: 2634.897
PCM Chl fluor sum: 56.2056
PCM Beam trans sum: 15602.546700000003
PCM O2 Mm/Kg sum: 7708.543999999999
DCM Chl fluor sum: 0.0
DCM Beam trans sum: 0.0
DCM O2 Mm/Kg sum: 0.0


In [21]:
# read from cnv with 'ignore' added to depSM
cast43 = ctd.from_cnv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/KM1919-St20-P2/St20_P2_cast43-align-cellTM-derive-bin.cnv")

# set up datetime to datenum 
ST_43 = pd.to_datetime(st43)

# split up and down casts; we want the downcast for the profile
down43, up43 = cast43.split()

# add a column with the start date and time in UTC (this is from the cnv header or from ship's log)
down43.insert(0, 'StartTime', pd.to_datetime(st43).replace(microsecond=0))

# slice the mixed layer out and save as csv
p2_43_ml = down43[down43['sigma-�00'].between(22.0,23.2)]
p2_43_ml.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/mixed_layer//Station20_cast43_ML.csv")

# get the sum of mixed layer slice oxygen, beam transmission, and chlorophyll fluor
cf_sum_p2_43_ml = p2_43_ml['flECO-AFL'].sum()
print('ML Chl fluor sum:', cf_sum_p2_43_ml)

bt_sum_p2_43_ml = p2_43_ml['CStarTr0'].sum()
print('ML Beam trans sum:', bt_sum_p2_43_ml)

o2_sum_p2_43_ml = p2_43_ml['sbeox0Mm/L'].sum()
print('ML O2 Mm/Kg sum:', o2_sum_p2_43_ml)

# Make lists of the input for the ML integrated dataframe
ML_data_43 = {'Cast':[43], 'StartTime':st43, 'StartTime matlab datenum': datetime_to_datenum(ST_43), \
           'Int chl fluor': cf_sum_p2_43_ml, \
           'Int beam trans': bt_sum_p2_43_ml, \
            'Int O2 Mm/Kg': o2_sum_p2_43_ml}

# slice the primary chlorophyll max out and save as csv
p2_43_pcm = down43[down43['sigma-�00'].between(23.2,43.7)]
p2_43_pcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/primary_chl/Station20_cast43_PCM.csv")

# get the sum of primary chl slice oxygen and beam transmission
cf_sum_p2_43_pcm = p2_43_pcm['flECO-AFL'].sum()
print('PCM Chl fluor sum:', cf_sum_p2_43_pcm)

bt_sum_p2_43_pcm = p2_43_pcm['CStarTr0'].sum()
print('PCM Beam trans sum:', bt_sum_p2_43_pcm)

o2_sum_p2_43_pcm = p2_43_pcm['sbeox0Mm/L'].sum()
print('PCM O2 Mm/Kg sum:', o2_sum_p2_43_pcm)

# Make lists of the input for the PCM integrated dataframe
PCM_data_43 = {'Cast':[43], 'StartTime':st43, 'StartTime matlab datenum': datetime_to_datenum(ST_43), \
            'Int chl fluor': cf_sum_p2_43_pcm, \
           'Int beam trans': bt_sum_p2_43_pcm, \
            'Int O2 Mm/Kg': o2_sum_p2_43_pcm}

# slice the deep chlorophyll max out and save as csv
p2_43_dcm = down43[down43['sigma-�00'].between(43.7,26.2)]
p2_43_dcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/deep_chl/Station20_cast43_DCM.csv")

# get the sum of deep chl slice oxygen and beam transmission
cf_sum_p2_43_dcm = p2_43_dcm['flECO-AFL'].sum()
print('DCM Chl fluor sum:', cf_sum_p2_43_dcm)

bt_sum_p2_43_dcm = p2_43_dcm['CStarTr0'].sum()
print('DCM Beam trans sum:', bt_sum_p2_43_dcm)

o2_sum_p2_43_dcm = p2_43_dcm['sbeox0Mm/L'].sum()
print('DCM O2 Mm/Kg sum:', o2_sum_p2_43_dcm)

# Make lists of the input for the DCM integrated dataframe
DCM_data_43 = {'Cast':[43], 'StartTime':st43, 'StartTime matlab datenum': datetime_to_datenum(ST_43), \
            'Int chl fluor': cf_sum_p2_43_dcm, \
           'Int beam trans': bt_sum_p2_43_dcm, \
            'Int O2 Mm/Kg': o2_sum_p2_43_dcm}

ML Chl fluor sum: 2.3507
ML Beam trans sum: 980.4527
ML O2 Mm/Kg sum: 2180.0820000000003
PCM Chl fluor sum: 57.0012
PCM Beam trans sum: 15208.3099
PCM O2 Mm/Kg sum: 7847.3240000000005
DCM Chl fluor sum: 0.0
DCM Beam trans sum: 0.0
DCM O2 Mm/Kg sum: 0.0


In [22]:
# read from cnv with 'ignore' added to depSM
cast44 = ctd.from_cnv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/KM1919-St20-P2/St20_P2_cast44-align-cellTM-derive-bin.cnv")

# set up datetime to datenum 
ST_44 = pd.to_datetime(st44)

# split up and down casts; we want the downcast for the profile
down44, up44 = cast44.split()

# add a column with the start date and time in UTC (this is from the cnv header or from ship's log)
down44.insert(0, 'StartTime', pd.to_datetime(st44).replace(microsecond=0))

# slice the mixed layer out and save as csv
p2_44_ml = down44[down44['sigma-�00'].between(22.0,23.2)]
p2_44_ml.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/mixed_layer//Station20_cast44_ML.csv")

# get the sum of mixed layer slice oxygen, beam transmission, and chlorophyll fluor
cf_sum_p2_44_ml = p2_44_ml['flECO-AFL'].sum()
print('ML Chl fluor sum:', cf_sum_p2_44_ml)

bt_sum_p2_44_ml = p2_44_ml['CStarTr0'].sum()
print('ML Beam trans sum:', bt_sum_p2_44_ml)

o2_sum_p2_44_ml = p2_44_ml['sbeox0Mm/L'].sum()
print('ML O2 Mm/Kg sum:', o2_sum_p2_44_ml)

# Make lists of the input for the ML integrated dataframe
ML_data_44 = {'Cast':[44], 'StartTime':st44, 'StartTime matlab datenum': datetime_to_datenum(ST_44), \
           'Int chl fluor': cf_sum_p2_44_ml, \
           'Int beam trans': bt_sum_p2_44_ml, \
            'Int O2 Mm/Kg': o2_sum_p2_44_ml}

# slice the primary chlorophyll max out and save as csv
p2_44_pcm = down44[down44['sigma-�00'].between(23.2,44.7)]
p2_44_pcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/primary_chl/Station20_cast44_PCM.csv")

# get the sum of primary chl slice oxygen and beam transmission
cf_sum_p2_44_pcm = p2_44_pcm['flECO-AFL'].sum()
print('PCM Chl fluor sum:', cf_sum_p2_44_pcm)

bt_sum_p2_44_pcm = p2_44_pcm['CStarTr0'].sum()
print('PCM Beam trans sum:', bt_sum_p2_44_pcm)

o2_sum_p2_44_pcm = p2_44_pcm['sbeox0Mm/L'].sum()
print('PCM O2 Mm/Kg sum:', o2_sum_p2_44_pcm)

# Make lists of the input for the PCM integrated dataframe
PCM_data_44 = {'Cast':[44], 'StartTime':st44, 'StartTime matlab datenum': datetime_to_datenum(ST_44), \
            'Int chl fluor': cf_sum_p2_44_pcm, \
           'Int beam trans': bt_sum_p2_44_pcm, \
            'Int O2 Mm/Kg': o2_sum_p2_44_pcm}

# slice the deep chlorophyll max out and save as csv
p2_44_dcm = down44[down44['sigma-�00'].between(44.7,26.2)]
p2_44_dcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/deep_chl/Station20_cast44_DCM.csv")

# get the sum of deep chl slice oxygen and beam transmission
cf_sum_p2_44_dcm = p2_44_dcm['flECO-AFL'].sum()
print('DCM Chl fluor sum:', cf_sum_p2_44_dcm)

bt_sum_p2_44_dcm = p2_44_dcm['CStarTr0'].sum()
print('DCM Beam trans sum:', bt_sum_p2_44_dcm)

o2_sum_p2_44_dcm = p2_44_dcm['sbeox0Mm/L'].sum()
print('DCM O2 Mm/Kg sum:', o2_sum_p2_44_dcm)

# Make lists of the input for the DCM integrated dataframe
DCM_data_44 = {'Cast':[44], 'StartTime':st44, 'StartTime matlab datenum': datetime_to_datenum(ST_44), \
            'Int chl fluor': cf_sum_p2_44_dcm, \
           'Int beam trans': bt_sum_p2_44_dcm, \
            'Int O2 Mm/Kg': o2_sum_p2_44_dcm}

ML Chl fluor sum: 3.3724000000000003
ML Beam trans sum: 1371.9504
ML O2 Mm/Kg sum: 3059.9730000000004
PCM Chl fluor sum: 50.1083
PCM Beam trans sum: 14615.4372
PCM O2 Mm/Kg sum: 7220.336
DCM Chl fluor sum: 0.0
DCM Beam trans sum: 0.0
DCM O2 Mm/Kg sum: 0.0


In [23]:
# read from cnv with 'ignore' added to depSM
cast45 = ctd.from_cnv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/KM1919-St20-P2/St20_P2_cast45-align-cellTM-derive-bin.cnv")

# set up datetime to datenum 
ST_45 = pd.to_datetime(st45)

# split up and down casts; we want the downcast for the profile
down45, up45 = cast45.split()

# add a column with the start date and time in UTC (this is from the cnv header or from ship's log)
down45.insert(0, 'StartTime', pd.to_datetime(st45).replace(microsecond=0))

# slice the mixed layer out and save as csv
p2_45_ml = down45[down45['sigma-�00'].between(22.0,23.2)]
p2_45_ml.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/mixed_layer//Station20_cast45_ML.csv")

# get the sum of mixed layer slice oxygen, beam transmission, and chlorophyll fluor
cf_sum_p2_45_ml = p2_45_ml['flECO-AFL'].sum()
print('ML Chl fluor sum:', cf_sum_p2_45_ml)

bt_sum_p2_45_ml = p2_45_ml['CStarTr0'].sum()
print('ML Beam trans sum:', bt_sum_p2_45_ml)

o2_sum_p2_45_ml = p2_45_ml['sbeox0Mm/L'].sum()
print('ML O2 Mm/Kg sum:', o2_sum_p2_45_ml)

# Make lists of the input for the ML integrated dataframe
ML_data_45 = {'Cast':[45], 'StartTime':st45, 'StartTime matlab datenum': datetime_to_datenum(ST_45), \
           'Int chl fluor': cf_sum_p2_45_ml, \
           'Int beam trans': bt_sum_p2_45_ml, \
            'Int O2 Mm/Kg': o2_sum_p2_45_ml}

# slice the primary chlorophyll max out and save as csv
p2_45_pcm = down45[down45['sigma-�00'].between(23.2,45.7)]
p2_45_pcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/primary_chl/Station20_cast45_PCM.csv")

# get the sum of primary chl slice oxygen and beam transmission
cf_sum_p2_45_pcm = p2_45_pcm['flECO-AFL'].sum()
print('PCM Chl fluor sum:', cf_sum_p2_45_pcm)

bt_sum_p2_45_pcm = p2_45_pcm['CStarTr0'].sum()
print('PCM Beam trans sum:', bt_sum_p2_45_pcm)

o2_sum_p2_45_pcm = p2_45_pcm['sbeox0Mm/L'].sum()
print('PCM O2 Mm/Kg sum:', o2_sum_p2_45_pcm)

# Make lists of the input for the PCM integrated dataframe
PCM_data_45 = {'Cast':[45], 'StartTime':st45, 'StartTime matlab datenum': datetime_to_datenum(ST_45), \
            'Int chl fluor': cf_sum_p2_45_pcm, \
           'Int beam trans': bt_sum_p2_45_pcm, \
            'Int O2 Mm/Kg': o2_sum_p2_45_pcm}

# slice the deep chlorophyll max out and save as csv
p2_45_dcm = down45[down45['sigma-�00'].between(45.7,26.2)]
p2_45_dcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/deep_chl/Station20_cast45_DCM.csv")

# get the sum of deep chl slice oxygen and beam transmission
cf_sum_p2_45_dcm = p2_45_dcm['flECO-AFL'].sum()
print('DCM Chl fluor sum:', cf_sum_p2_45_dcm)

bt_sum_p2_45_dcm = p2_45_dcm['CStarTr0'].sum()
print('DCM Beam trans sum:', bt_sum_p2_45_dcm)

o2_sum_p2_45_dcm = p2_45_dcm['sbeox0Mm/L'].sum()
print('DCM O2 Mm/Kg sum:', o2_sum_p2_45_dcm)

# Make lists of the input for the DCM integrated dataframe
DCM_data_45 = {'Cast':[45], 'StartTime':st45, 'StartTime matlab datenum': datetime_to_datenum(ST_45), \
            'Int chl fluor': cf_sum_p2_45_dcm, \
           'Int beam trans': bt_sum_p2_45_dcm, \
            'Int O2 Mm/Kg': o2_sum_p2_45_dcm}

ML Chl fluor sum: 1.8176999999999999
ML Beam trans sum: 782.5113
ML O2 Mm/Kg sum: 1779.9009999999998
PCM Chl fluor sum: 60.6235
PCM Beam trans sum: 15791.928899999999
PCM O2 Mm/Kg sum: 9070.543
DCM Chl fluor sum: 0.0
DCM Beam trans sum: 0.0
DCM O2 Mm/Kg sum: 0.0


In [24]:
# read from cnv with 'ignore' added to depSM
cast46 = ctd.from_cnv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/KM1919-St20-P2/St20_P2_cast46-align-cellTM-derive-bin.cnv")

# set up datetime to datenum 
ST_46 = pd.to_datetime(st46)

# split up and down casts; we want the downcast for the profile
down46, up46 = cast46.split()

# add a column with the start date and time in UTC (this is from the cnv header or from ship's log)
down46.insert(0, 'StartTime', pd.to_datetime(st46).replace(microsecond=0))

# slice the mixed layer out and save as csv
p2_46_ml = down46[down46['sigma-�00'].between(22.0,23.2)]
p2_46_ml.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/mixed_layer//Station20_cast46_ML.csv")

# get the sum of mixed layer slice oxygen, beam transmission, and chlorophyll fluor
cf_sum_p2_46_ml = p2_46_ml['flECO-AFL'].sum()
print('ML Chl fluor sum:', cf_sum_p2_46_ml)

bt_sum_p2_46_ml = p2_46_ml['CStarTr0'].sum()
print('ML Beam trans sum:', bt_sum_p2_46_ml)

o2_sum_p2_46_ml = p2_46_ml['sbeox0Mm/L'].sum()
print('ML O2 Mm/Kg sum:', o2_sum_p2_46_ml)

# Make lists of the input for the ML integrated dataframe
ML_data_46 = {'Cast':[46], 'StartTime':st46, 'StartTime matlab datenum': datetime_to_datenum(ST_46), \
           'Int chl fluor': cf_sum_p2_46_ml, \
           'Int beam trans': bt_sum_p2_46_ml, \
            'Int O2 Mm/Kg': o2_sum_p2_46_ml}

# slice the primary chlorophyll max out and save as csv
p2_46_pcm = down46[down46['sigma-�00'].between(23.2,46.7)]
p2_46_pcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/primary_chl/Station20_cast46_PCM.csv")

# get the sum of primary chl slice oxygen and beam transmission
cf_sum_p2_46_pcm = p2_46_pcm['flECO-AFL'].sum()
print('PCM Chl fluor sum:', cf_sum_p2_46_pcm)

bt_sum_p2_46_pcm = p2_46_pcm['CStarTr0'].sum()
print('PCM Beam trans sum:', bt_sum_p2_46_pcm)

o2_sum_p2_46_pcm = p2_46_pcm['sbeox0Mm/L'].sum()
print('PCM O2 Mm/Kg sum:', o2_sum_p2_46_pcm)

# Make lists of the input for the PCM integrated dataframe
PCM_data_46 = {'Cast':[46], 'StartTime':st46, 'StartTime matlab datenum': datetime_to_datenum(ST_46), \
            'Int chl fluor': cf_sum_p2_46_pcm, \
           'Int beam trans': bt_sum_p2_46_pcm, \
            'Int O2 Mm/Kg': o2_sum_p2_46_pcm}

# slice the deep chlorophyll max out and save as csv
p2_46_dcm = down46[down46['sigma-�00'].between(46.7,26.2)]
p2_46_dcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/deep_chl/Station20_cast46_DCM.csv")

# get the sum of deep chl slice oxygen and beam transmission
cf_sum_p2_46_dcm = p2_46_dcm['flECO-AFL'].sum()
print('DCM Chl fluor sum:', cf_sum_p2_46_dcm)

bt_sum_p2_46_dcm = p2_46_dcm['CStarTr0'].sum()
print('DCM Beam trans sum:', bt_sum_p2_46_dcm)

o2_sum_p2_46_dcm = p2_46_dcm['sbeox0Mm/L'].sum()
print('DCM O2 Mm/Kg sum:', o2_sum_p2_46_dcm)

# Make lists of the input for the DCM integrated dataframe
DCM_data_46 = {'Cast':[46], 'StartTime':st46, 'StartTime matlab datenum': datetime_to_datenum(ST_46), \
            'Int chl fluor': cf_sum_p2_46_dcm, \
           'Int beam trans': bt_sum_p2_46_dcm, \
            'Int O2 Mm/Kg': o2_sum_p2_46_dcm}

ML Chl fluor sum: 2.2252
ML Beam trans sum: 880.2536
ML O2 Mm/Kg sum: 1985.226
PCM Chl fluor sum: 70.05799999999999
PCM Beam trans sum: 35495.5205
PCM O2 Mm/Kg sum: 7997.861
DCM Chl fluor sum: 0.0
DCM Beam trans sum: 0.0
DCM O2 Mm/Kg sum: 0.0


In [25]:
# read from cnv with 'ignore' added to depSM
cast47 = ctd.from_cnv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/KM1919-St20-P2/St20_P2_cast47-align-cellTM-derive-bin.cnv")

# set up datetime to datenum 
ST_47 = pd.to_datetime(st47)

# split up and down casts; we want the downcast for the profile
down47, up47 = cast47.split()

# add a column with the start date and time in UTC (this is from the cnv header or from ship's log)
down47.insert(0, 'StartTime', pd.to_datetime(st47).replace(microsecond=0))

# slice the mixed layer out and save as csv
p2_47_ml = down47[down47['sigma-�00'].between(22.0,23.2)]
p2_47_ml.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/mixed_layer//Station20_cast47_ML.csv")

# get the sum of mixed layer slice oxygen, beam transmission, and chlorophyll fluor
cf_sum_p2_47_ml = p2_47_ml['flECO-AFL'].sum()
print('ML Chl fluor sum:', cf_sum_p2_47_ml)

bt_sum_p2_47_ml = p2_47_ml['CStarTr0'].sum()
print('ML Beam trans sum:', bt_sum_p2_47_ml)

o2_sum_p2_47_ml = p2_47_ml['sbeox0Mm/L'].sum()
print('ML O2 Mm/Kg sum:', o2_sum_p2_47_ml)

# Make lists of the input for the ML integrated dataframe
ML_data_47 = {'Cast':[47], 'StartTime':st47, 'StartTime matlab datenum': datetime_to_datenum(ST_47), \
           'Int chl fluor': cf_sum_p2_47_ml, \
           'Int beam trans': bt_sum_p2_47_ml, \
            'Int O2 Mm/Kg': o2_sum_p2_47_ml}

# slice the primary chlorophyll max out and save as csv
p2_47_pcm = down47[down47['sigma-�00'].between(23.2,47.7)]
p2_47_pcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/primary_chl/Station20_cast47_PCM.csv")

# get the sum of primary chl slice oxygen and beam transmission
cf_sum_p2_47_pcm = p2_47_pcm['flECO-AFL'].sum()
print('PCM Chl fluor sum:', cf_sum_p2_47_pcm)

bt_sum_p2_47_pcm = p2_47_pcm['CStarTr0'].sum()
print('PCM Beam trans sum:', bt_sum_p2_47_pcm)

o2_sum_p2_47_pcm = p2_47_pcm['sbeox0Mm/L'].sum()
print('PCM O2 Mm/Kg sum:', o2_sum_p2_47_pcm)

# Make lists of the input for the PCM integrated dataframe
PCM_data_47 = {'Cast':[47], 'StartTime':st47, 'StartTime matlab datenum': datetime_to_datenum(ST_47), \
            'Int chl fluor': cf_sum_p2_47_pcm, \
           'Int beam trans': bt_sum_p2_47_pcm, \
            'Int O2 Mm/Kg': o2_sum_p2_47_pcm}

# slice the deep chlorophyll max out and save as csv
p2_47_dcm = down47[down47['sigma-�00'].between(47.7,26.2)]
p2_47_dcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/deep_chl/Station20_cast47_DCM.csv")

# get the sum of deep chl slice oxygen and beam transmission
cf_sum_p2_47_dcm = p2_47_dcm['flECO-AFL'].sum()
print('DCM Chl fluor sum:', cf_sum_p2_47_dcm)

bt_sum_p2_47_dcm = p2_47_dcm['CStarTr0'].sum()
print('DCM Beam trans sum:', bt_sum_p2_47_dcm)

o2_sum_p2_47_dcm = p2_47_dcm['sbeox0Mm/L'].sum()
print('DCM O2 Mm/Kg sum:', o2_sum_p2_47_dcm)

# Make lists of the input for the DCM integrated dataframe
DCM_data_47 = {'Cast':[47], 'StartTime':st47, 'StartTime matlab datenum': datetime_to_datenum(ST_47), \
            'Int chl fluor': cf_sum_p2_47_dcm, \
           'Int beam trans': bt_sum_p2_47_dcm, \
            'Int O2 Mm/Kg': o2_sum_p2_47_dcm}

ML Chl fluor sum: 3.9096
ML Beam trans sum: 1467.7521000000002
ML O2 Mm/Kg sum: 3300.8689999999997
PCM Chl fluor sum: 57.2566
PCM Beam trans sum: 14923.983499999998
PCM O2 Mm/Kg sum: 7953.786
DCM Chl fluor sum: 0.0
DCM Beam trans sum: 0.0
DCM O2 Mm/Kg sum: 0.0


In [26]:
# read from cnv with 'ignore' added to depSM
cast48 = ctd.from_cnv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/KM1919-St20-P2/St20_P2_cast48-align-cellTM-derive-bin.cnv")

# set up datetime to datenum 
ST_48 = pd.to_datetime(st48)

# split up and down casts; we want the downcast for the profile
down48, up48 = cast48.split()

# add a column with the start date and time in UTC (this is from the cnv header or from ship's log)
down48.insert(0, 'StartTime', pd.to_datetime(st48).replace(microsecond=0))

# slice the mixed layer out and save as csv
p2_48_ml = down48[down48['sigma-�00'].between(22.0,23.2)]
p2_48_ml.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/mixed_layer//Station20_cast48_ML.csv")

# get the sum of mixed layer slice oxygen, beam transmission, and chlorophyll fluor
cf_sum_p2_48_ml = p2_48_ml['flECO-AFL'].sum()
print('ML Chl fluor sum:', cf_sum_p2_48_ml)

bt_sum_p2_48_ml = p2_48_ml['CStarTr0'].sum()
print('ML Beam trans sum:', bt_sum_p2_48_ml)

o2_sum_p2_48_ml = p2_48_ml['sbeox0Mm/L'].sum()
print('ML O2 Mm/Kg sum:', o2_sum_p2_48_ml)

# Make lists of the input for the ML integrated dataframe
ML_data_48 = {'Cast':[48], 'StartTime':st48, 'StartTime matlab datenum': datetime_to_datenum(ST_48), \
           'Int chl fluor': cf_sum_p2_48_ml, \
           'Int beam trans': bt_sum_p2_48_ml, \
            'Int O2 Mm/Kg': o2_sum_p2_48_ml}

# slice the primary chlorophyll max out and save as csv
p2_48_pcm = down48[down48['sigma-�00'].between(23.2,48.7)]
p2_48_pcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/primary_chl/Station20_cast48_PCM.csv")

# get the sum of primary chl slice oxygen and beam transmission
cf_sum_p2_48_pcm = p2_48_pcm['flECO-AFL'].sum()
print('PCM Chl fluor sum:', cf_sum_p2_48_pcm)

bt_sum_p2_48_pcm = p2_48_pcm['CStarTr0'].sum()
print('PCM Beam trans sum:', bt_sum_p2_48_pcm)

o2_sum_p2_48_pcm = p2_48_pcm['sbeox0Mm/L'].sum()
print('PCM O2 Mm/Kg sum:', o2_sum_p2_48_pcm)

# Make lists of the input for the PCM integrated dataframe
PCM_data_48 = {'Cast':[48], 'StartTime':st48, 'StartTime matlab datenum': datetime_to_datenum(ST_48), \
            'Int chl fluor': cf_sum_p2_48_pcm, \
           'Int beam trans': bt_sum_p2_48_pcm, \
            'Int O2 Mm/Kg': o2_sum_p2_48_pcm}

# slice the deep chlorophyll max out and save as csv
p2_48_dcm = down48[down48['sigma-�00'].between(48.7,26.2)]
p2_48_dcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/deep_chl/Station20_cast48_DCM.csv")

# get the sum of deep chl slice oxygen and beam transmission
cf_sum_p2_48_dcm = p2_48_dcm['flECO-AFL'].sum()
print('DCM Chl fluor sum:', cf_sum_p2_48_dcm)

bt_sum_p2_48_dcm = p2_48_dcm['CStarTr0'].sum()
print('DCM Beam trans sum:', bt_sum_p2_48_dcm)

o2_sum_p2_48_dcm = p2_48_dcm['sbeox0Mm/L'].sum()
print('DCM O2 Mm/Kg sum:', o2_sum_p2_48_dcm)

# Make lists of the input for the DCM integrated dataframe
DCM_data_48 = {'Cast':[48], 'StartTime':st48, 'StartTime matlab datenum': datetime_to_datenum(ST_48), \
            'Int chl fluor': cf_sum_p2_48_dcm, \
           'Int beam trans': bt_sum_p2_48_dcm, \
            'Int O2 Mm/Kg': o2_sum_p2_48_dcm}

ML Chl fluor sum: 3.6037000000000003
ML Beam trans sum: 1372.725
ML O2 Mm/Kg sum: 3096.0589999999997
PCM Chl fluor sum: 67.934
PCM Beam trans sum: 30441.8505
PCM O2 Mm/Kg sum: 7641.129999999999
DCM Chl fluor sum: 0.0
DCM Beam trans sum: 0.0
DCM O2 Mm/Kg sum: 0.0


In [27]:
# read from cnv with 'ignore' added to depSM
cast49 = ctd.from_cnv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/KM1919-St20-P2/St20_P2_cast49-align-cellTM-derive-bin.cnv")

# set up datetime to datenum 
ST_49 = pd.to_datetime(st49)

# split up and down casts; we want the downcast for the profile
down49, up49 = cast49.split()

# add a column with the start date and time in UTC (this is from the cnv header or from ship's log)
down49.insert(0, 'StartTime', pd.to_datetime(st49).replace(microsecond=0))

# slice the mixed layer out and save as csv
p2_49_ml = down49[down49['sigma-�00'].between(22.0,23.2)]
p2_49_ml.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/mixed_layer//Station20_cast49_ML.csv")

# get the sum of mixed layer slice oxygen, beam transmission, and chlorophyll fluor
cf_sum_p2_49_ml = p2_49_ml['flECO-AFL'].sum()
print('ML Chl fluor sum:', cf_sum_p2_49_ml)

bt_sum_p2_49_ml = p2_49_ml['CStarTr0'].sum()
print('ML Beam trans sum:', bt_sum_p2_49_ml)

o2_sum_p2_49_ml = p2_49_ml['sbeox0Mm/L'].sum()
print('ML O2 Mm/Kg sum:', o2_sum_p2_49_ml)

# Make lists of the input for the ML integrated dataframe
ML_data_49 = {'Cast':[49], 'StartTime':st49, 'StartTime matlab datenum': datetime_to_datenum(ST_49), \
           'Int chl fluor': cf_sum_p2_49_ml, \
           'Int beam trans': bt_sum_p2_49_ml, \
            'Int O2 Mm/Kg': o2_sum_p2_49_ml}

# slice the primary chlorophyll max out and save as csv
p2_49_pcm = down49[down49['sigma-�00'].between(23.2,49.7)]
p2_49_pcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/primary_chl/Station20_cast49_PCM.csv")

# get the sum of primary chl slice oxygen and beam transmission
cf_sum_p2_49_pcm = p2_49_pcm['flECO-AFL'].sum()
print('PCM Chl fluor sum:', cf_sum_p2_49_pcm)

bt_sum_p2_49_pcm = p2_49_pcm['CStarTr0'].sum()
print('PCM Beam trans sum:', bt_sum_p2_49_pcm)

o2_sum_p2_49_pcm = p2_49_pcm['sbeox0Mm/L'].sum()
print('PCM O2 Mm/Kg sum:', o2_sum_p2_49_pcm)

# Make lists of the input for the PCM integrated dataframe
PCM_data_49 = {'Cast':[49], 'StartTime':st49, 'StartTime matlab datenum': datetime_to_datenum(ST_49), \
            'Int chl fluor': cf_sum_p2_49_pcm, \
           'Int beam trans': bt_sum_p2_49_pcm, \
            'Int O2 Mm/Kg': o2_sum_p2_49_pcm}

# slice the deep chlorophyll max out and save as csv
p2_49_dcm = down49[down49['sigma-�00'].between(49.7,26.2)]
p2_49_dcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/deep_chl/Station20_cast49_DCM.csv")

# get the sum of deep chl slice oxygen and beam transmission
cf_sum_p2_49_dcm = p2_49_dcm['flECO-AFL'].sum()
print('DCM Chl fluor sum:', cf_sum_p2_49_dcm)

bt_sum_p2_49_dcm = p2_49_dcm['CStarTr0'].sum()
print('DCM Beam trans sum:', bt_sum_p2_49_dcm)

o2_sum_p2_49_dcm = p2_49_dcm['sbeox0Mm/L'].sum()
print('DCM O2 Mm/Kg sum:', o2_sum_p2_49_dcm)

# Make lists of the input for the DCM integrated dataframe
DCM_data_49 = {'Cast':[49], 'StartTime':st49, 'StartTime matlab datenum': datetime_to_datenum(ST_49), \
            'Int chl fluor': cf_sum_p2_49_dcm, \
           'Int beam trans': bt_sum_p2_49_dcm, \
            'Int O2 Mm/Kg': o2_sum_p2_49_dcm}

ML Chl fluor sum: 2.8391
ML Beam trans sum: 1177.8184999999999
ML O2 Mm/Kg sum: 2636.4240000000004
PCM Chl fluor sum: 67.652
PCM Beam trans sum: 34926.523199999996
PCM O2 Mm/Kg sum: 6535.573
DCM Chl fluor sum: 0.0
DCM Beam trans sum: 0.0
DCM O2 Mm/Kg sum: 0.0


In [28]:
# read from cnv with 'ignore' added to depSM
cast50 = ctd.from_cnv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/KM1919-St20-P2/St20_P2_cast50-align-cellTM-derive-bin.cnv")

# set up datetime to datenum 
ST_50 = pd.to_datetime(st50)

# split up and down casts; we want the downcast for the profile
down50, up50 = cast50.split()

# add a column with the start date and time in UTC (this is from the cnv header or from ship's log)
down50.insert(0, 'StartTime', pd.to_datetime(st50).replace(microsecond=0))

# slice the mixed layer out and save as csv
p2_50_ml = down50[down50['sigma-�00'].between(22.0,23.2)]
p2_50_ml.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/mixed_layer//Station20_cast50_ML.csv")

# get the sum of mixed layer slice oxygen, beam transmission, and chlorophyll fluor
cf_sum_p2_50_ml = p2_50_ml['flECO-AFL'].sum()
print('ML Chl fluor sum:', cf_sum_p2_50_ml)

bt_sum_p2_50_ml = p2_50_ml['CStarTr0'].sum()
print('ML Beam trans sum:', bt_sum_p2_50_ml)

o2_sum_p2_50_ml = p2_50_ml['sbeox0Mm/L'].sum()
print('ML O2 Mm/Kg sum:', o2_sum_p2_50_ml)

# Make lists of the input for the ML integrated dataframe
ML_data_50 = {'Cast':[50], 'StartTime':st50, 'StartTime matlab datenum': datetime_to_datenum(ST_50), \
           'Int chl fluor': cf_sum_p2_50_ml, \
           'Int beam trans': bt_sum_p2_50_ml, \
            'Int O2 Mm/Kg': o2_sum_p2_50_ml}

# slice the primary chlorophyll max out and save as csv
p2_50_pcm = down50[down50['sigma-�00'].between(23.2,50.7)]
p2_50_pcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/primary_chl/Station20_cast50_PCM.csv")

# get the sum of primary chl slice oxygen and beam transmission
cf_sum_p2_50_pcm = p2_50_pcm['flECO-AFL'].sum()
print('PCM Chl fluor sum:', cf_sum_p2_50_pcm)

bt_sum_p2_50_pcm = p2_50_pcm['CStarTr0'].sum()
print('PCM Beam trans sum:', bt_sum_p2_50_pcm)

o2_sum_p2_50_pcm = p2_50_pcm['sbeox0Mm/L'].sum()
print('PCM O2 Mm/Kg sum:', o2_sum_p2_50_pcm)

# Make lists of the input for the PCM integrated dataframe
PCM_data_50 = {'Cast':[50], 'StartTime':st50, 'StartTime matlab datenum': datetime_to_datenum(ST_50), \
            'Int chl fluor': cf_sum_p2_50_pcm, \
           'Int beam trans': bt_sum_p2_50_pcm, \
            'Int O2 Mm/Kg': o2_sum_p2_50_pcm}

# slice the deep chlorophyll max out and save as csv
p2_50_dcm = down50[down50['sigma-�00'].between(50.7,26.2)]
p2_50_dcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/deep_chl/Station20_cast50_DCM.csv")

# get the sum of deep chl slice oxygen and beam transmission
cf_sum_p2_50_dcm = p2_50_dcm['flECO-AFL'].sum()
print('DCM Chl fluor sum:', cf_sum_p2_50_dcm)

bt_sum_p2_50_dcm = p2_50_dcm['CStarTr0'].sum()
print('DCM Beam trans sum:', bt_sum_p2_50_dcm)

o2_sum_p2_50_dcm = p2_50_dcm['sbeox0Mm/L'].sum()
print('DCM O2 Mm/Kg sum:', o2_sum_p2_50_dcm)

# Make lists of the input for the DCM integrated dataframe
DCM_data_50 = {'Cast':[50], 'StartTime':st50, 'StartTime matlab datenum': datetime_to_datenum(ST_50), \
            'Int chl fluor': cf_sum_p2_50_dcm, \
           'Int beam trans': bt_sum_p2_50_dcm, \
            'Int O2 Mm/Kg': o2_sum_p2_50_dcm}

ML Chl fluor sum: 3.9346
ML Beam trans sum: 1472.2893000000001
ML O2 Mm/Kg sum: 3322.799
PCM Chl fluor sum: 59.50959999999999
PCM Beam trans sum: 14929.393199999999
PCM O2 Mm/Kg sum: 8103.999
DCM Chl fluor sum: 0.0
DCM Beam trans sum: 0.0
DCM O2 Mm/Kg sum: 0.0


In [29]:
# read from cnv with 'ignore' added to depSM
cast51 = ctd.from_cnv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/KM1919-St20-P2/St20_P2_cast51-align-cellTM-derive-bin.cnv")

# set up datetime to datenum 
ST_51 = pd.to_datetime(st51)

# split up and down casts; we want the downcast for the profile
down51, up51 = cast51.split()

# add a column with the start date and time in UTC (this is from the cnv header or from ship's log)
down51.insert(0, 'StartTime', pd.to_datetime(st51).replace(microsecond=0))

# slice the mixed layer out and save as csv
p2_51_ml = down51[down51['sigma-�00'].between(22.0,23.2)]
p2_51_ml.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/mixed_layer//Station20_cast51_ML.csv")

# get the sum of mixed layer slice oxygen, beam transmission, and chlorophyll fluor
cf_sum_p2_51_ml = p2_51_ml['flECO-AFL'].sum()
print('ML Chl fluor sum:', cf_sum_p2_51_ml)

bt_sum_p2_51_ml = p2_51_ml['CStarTr0'].sum()
print('ML Beam trans sum:', bt_sum_p2_51_ml)

o2_sum_p2_51_ml = p2_51_ml['sbeox0Mm/L'].sum()
print('ML O2 Mm/Kg sum:', o2_sum_p2_51_ml)

# Make lists of the input for the ML integrated dataframe
ML_data_51 = {'Cast':[51], 'StartTime':st51, 'StartTime matlab datenum': datetime_to_datenum(ST_51), \
           'Int chl fluor': cf_sum_p2_51_ml, \
           'Int beam trans': bt_sum_p2_51_ml, \
            'Int O2 Mm/Kg': o2_sum_p2_51_ml}

# slice the primary chlorophyll max out and save as csv
p2_51_pcm = down51[down51['sigma-�00'].between(23.2,51.7)]
p2_51_pcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/primary_chl/Station20_cast51_PCM.csv")

# get the sum of primary chl slice oxygen and beam transmission
cf_sum_p2_51_pcm = p2_51_pcm['flECO-AFL'].sum()
print('PCM Chl fluor sum:', cf_sum_p2_51_pcm)

bt_sum_p2_51_pcm = p2_51_pcm['CStarTr0'].sum()
print('PCM Beam trans sum:', bt_sum_p2_51_pcm)

o2_sum_p2_51_pcm = p2_51_pcm['sbeox0Mm/L'].sum()
print('PCM O2 Mm/Kg sum:', o2_sum_p2_51_pcm)

# Make lists of the input for the PCM integrated dataframe
PCM_data_51 = {'Cast':[51], 'StartTime':st51, 'StartTime matlab datenum': datetime_to_datenum(ST_51), \
            'Int chl fluor': cf_sum_p2_51_pcm, \
           'Int beam trans': bt_sum_p2_51_pcm, \
            'Int O2 Mm/Kg': o2_sum_p2_51_pcm}

# slice the deep chlorophyll max out and save as csv
p2_51_dcm = down51[down51['sigma-�00'].between(51.7,26.2)]
p2_51_dcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/deep_chl/Station20_cast51_DCM.csv")

# get the sum of deep chl slice oxygen and beam transmission
cf_sum_p2_51_dcm = p2_51_dcm['flECO-AFL'].sum()
print('DCM Chl fluor sum:', cf_sum_p2_51_dcm)

bt_sum_p2_51_dcm = p2_51_dcm['CStarTr0'].sum()
print('DCM Beam trans sum:', bt_sum_p2_51_dcm)

o2_sum_p2_51_dcm = p2_51_dcm['sbeox0Mm/L'].sum()
print('DCM O2 Mm/Kg sum:', o2_sum_p2_51_dcm)

# Make lists of the input for the DCM integrated dataframe
DCM_data_51 = {'Cast':[51], 'StartTime':st51, 'StartTime matlab datenum': datetime_to_datenum(ST_51), \
            'Int chl fluor': cf_sum_p2_51_dcm, \
           'Int beam trans': bt_sum_p2_51_dcm, \
            'Int O2 Mm/Kg': o2_sum_p2_51_dcm}

ML Chl fluor sum: 3.218
ML Beam trans sum: 1174.5655
ML O2 Mm/Kg sum: 2650.5060000000003
PCM Chl fluor sum: 60.56280000000001
PCM Beam trans sum: 15700.1971
PCM O2 Mm/Kg sum: 8376.274
DCM Chl fluor sum: 0.0
DCM Beam trans sum: 0.0
DCM O2 Mm/Kg sum: 0.0


In [30]:
# read from cnv with 'ignore' added to depSM
cast52 = ctd.from_cnv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/KM1919-St20-P2/St20_P2_cast52-align-cellTM-derive-bin.cnv")

# set up datetime to datenum 
ST_52 = pd.to_datetime(st52)

# split up and down casts; we want the downcast for the profile
down52, up52 = cast52.split()

# add a column with the start date and time in UTC (this is from the cnv header or from ship's log)
down52.insert(0, 'StartTime', pd.to_datetime(st52).replace(microsecond=0))

# slice the mixed layer out and save as csv
p2_52_ml = down52[down52['sigma-�00'].between(22.0,23.2)]
p2_52_ml.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/mixed_layer//Station20_cast52_ML.csv")

# get the sum of mixed layer slice oxygen, beam transmission, and chlorophyll fluor
cf_sum_p2_52_ml = p2_52_ml['flECO-AFL'].sum()
print('ML Chl fluor sum:', cf_sum_p2_52_ml)

bt_sum_p2_52_ml = p2_52_ml['CStarTr0'].sum()
print('ML Beam trans sum:', bt_sum_p2_52_ml)

o2_sum_p2_52_ml = p2_52_ml['sbeox0Mm/L'].sum()
print('ML O2 Mm/Kg sum:', o2_sum_p2_52_ml)

# Make lists of the input for the ML integrated dataframe
ML_data_52 = {'Cast':[52], 'StartTime':st52, 'StartTime matlab datenum': datetime_to_datenum(ST_52), \
           'Int chl fluor': cf_sum_p2_52_ml, \
           'Int beam trans': bt_sum_p2_52_ml, \
            'Int O2 Mm/Kg': o2_sum_p2_52_ml}

# slice the primary chlorophyll max out and save as csv
p2_52_pcm = down52[down52['sigma-�00'].between(23.2,52.7)]
p2_52_pcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/primary_chl/Station20_cast52_PCM.csv")

# get the sum of primary chl slice oxygen and beam transmission
cf_sum_p2_52_pcm = p2_52_pcm['flECO-AFL'].sum()
print('PCM Chl fluor sum:', cf_sum_p2_52_pcm)

bt_sum_p2_52_pcm = p2_52_pcm['CStarTr0'].sum()
print('PCM Beam trans sum:', bt_sum_p2_52_pcm)

o2_sum_p2_52_pcm = p2_52_pcm['sbeox0Mm/L'].sum()
print('PCM O2 Mm/Kg sum:', o2_sum_p2_52_pcm)

# Make lists of the input for the PCM integrated dataframe
PCM_data_52 = {'Cast':[52], 'StartTime':st52, 'StartTime matlab datenum': datetime_to_datenum(ST_52), \
            'Int chl fluor': cf_sum_p2_52_pcm, \
           'Int beam trans': bt_sum_p2_52_pcm, \
            'Int O2 Mm/Kg': o2_sum_p2_52_pcm}

# slice the deep chlorophyll max out and save as csv
p2_52_dcm = down52[down52['sigma-�00'].between(52.7,26.2)]
p2_52_dcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/deep_chl/Station20_cast52_DCM.csv")

# get the sum of deep chl slice oxygen and beam transmission
cf_sum_p2_52_dcm = p2_52_dcm['flECO-AFL'].sum()
print('DCM Chl fluor sum:', cf_sum_p2_52_dcm)

bt_sum_p2_52_dcm = p2_52_dcm['CStarTr0'].sum()
print('DCM Beam trans sum:', bt_sum_p2_52_dcm)

o2_sum_p2_52_dcm = p2_52_dcm['sbeox0Mm/L'].sum()
print('DCM O2 Mm/Kg sum:', o2_sum_p2_52_dcm)

# Make lists of the input for the DCM integrated dataframe
DCM_data_52 = {'Cast':[52], 'StartTime':st52, 'StartTime matlab datenum': datetime_to_datenum(ST_52), \
            'Int chl fluor': cf_sum_p2_52_dcm, \
           'Int beam trans': bt_sum_p2_52_dcm, \
            'Int O2 Mm/Kg': o2_sum_p2_52_dcm}

ML Chl fluor sum: 6.6882
ML Beam trans sum: 1662.2503
ML O2 Mm/Kg sum: 3809.292
PCM Chl fluor sum: 58.280699999999996
PCM Beam trans sum: 15490.6096
PCM O2 Mm/Kg sum: 8823.714
DCM Chl fluor sum: 0.0
DCM Beam trans sum: 0.0
DCM O2 Mm/Kg sum: 0.0


In [31]:
# read from cnv with 'ignore' added to depSM
cast53 = ctd.from_cnv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/KM1919-St20-P2/St20_P2_cast53-align-cellTM-derive-bin.cnv")

# set up datetime to datenum 
ST_53 = pd.to_datetime(st53)

# split up and down casts; we want the downcast for the profile
down53, up53 = cast53.split()

# add a column with the start date and time in UTC (this is from the cnv header or from ship's log)
down53.insert(0, 'StartTime', pd.to_datetime(st53).replace(microsecond=0))

# slice the mixed layer out and save as csv
p2_53_ml = down53[down53['sigma-�00'].between(22.0,23.2)]
p2_53_ml.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/mixed_layer//Station20_cast53_ML.csv")

# get the sum of mixed layer slice oxygen, beam transmission, and chlorophyll fluor
cf_sum_p2_53_ml = p2_53_ml['flECO-AFL'].sum()
print('ML Chl fluor sum:', cf_sum_p2_53_ml)

bt_sum_p2_53_ml = p2_53_ml['CStarTr0'].sum()
print('ML Beam trans sum:', bt_sum_p2_53_ml)

o2_sum_p2_53_ml = p2_53_ml['sbeox0Mm/L'].sum()
print('ML O2 Mm/Kg sum:', o2_sum_p2_53_ml)

# Make lists of the input for the ML integrated dataframe
ML_data_53 = {'Cast':[53], 'StartTime':st53, 'StartTime matlab datenum': datetime_to_datenum(ST_53), \
           'Int chl fluor': cf_sum_p2_53_ml, \
           'Int beam trans': bt_sum_p2_53_ml, \
            'Int O2 Mm/Kg': o2_sum_p2_53_ml}

# slice the primary chlorophyll max out and save as csv
p2_53_pcm = down53[down53['sigma-�00'].between(23.2,53.7)]
p2_53_pcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/primary_chl/Station20_cast53_PCM.csv")

# get the sum of primary chl slice oxygen and beam transmission
cf_sum_p2_53_pcm = p2_53_pcm['flECO-AFL'].sum()
print('PCM Chl fluor sum:', cf_sum_p2_53_pcm)

bt_sum_p2_53_pcm = p2_53_pcm['CStarTr0'].sum()
print('PCM Beam trans sum:', bt_sum_p2_53_pcm)

o2_sum_p2_53_pcm = p2_53_pcm['sbeox0Mm/L'].sum()
print('PCM O2 Mm/Kg sum:', o2_sum_p2_53_pcm)

# Make lists of the input for the PCM integrated dataframe
PCM_data_53 = {'Cast':[53], 'StartTime':st53, 'StartTime matlab datenum': datetime_to_datenum(ST_53), \
            'Int chl fluor': cf_sum_p2_53_pcm, \
           'Int beam trans': bt_sum_p2_53_pcm, \
            'Int O2 Mm/Kg': o2_sum_p2_53_pcm}

# slice the deep chlorophyll max out and save as csv
p2_53_dcm = down53[down53['sigma-�00'].between(53.7,26.2)]
p2_53_dcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/deep_chl/Station20_cast53_DCM.csv")

# get the sum of deep chl slice oxygen and beam transmission
cf_sum_p2_53_dcm = p2_53_dcm['flECO-AFL'].sum()
print('DCM Chl fluor sum:', cf_sum_p2_53_dcm)

bt_sum_p2_53_dcm = p2_53_dcm['CStarTr0'].sum()
print('DCM Beam trans sum:', bt_sum_p2_53_dcm)

o2_sum_p2_53_dcm = p2_53_dcm['sbeox0Mm/L'].sum()
print('DCM O2 Mm/Kg sum:', o2_sum_p2_53_dcm)

# Make lists of the input for the DCM integrated dataframe
DCM_data_53 = {'Cast':[53], 'StartTime':st53, 'StartTime matlab datenum': datetime_to_datenum(ST_53), \
            'Int chl fluor': cf_sum_p2_53_dcm, \
           'Int beam trans': bt_sum_p2_53_dcm, \
            'Int O2 Mm/Kg': o2_sum_p2_53_dcm}

ML Chl fluor sum: 4.035899999999999
ML Beam trans sum: 1173.468
ML O2 Mm/Kg sum: 2677.337
PCM Chl fluor sum: 55.14460000000001
PCM Beam trans sum: 15665.562000000002
PCM O2 Mm/Kg sum: 8159.375
DCM Chl fluor sum: 0.0
DCM Beam trans sum: 0.0
DCM O2 Mm/Kg sum: 0.0


In [32]:
# read from cnv with 'ignore' added to depSM
cast54 = ctd.from_cnv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/KM1919-St20-P2/St20_P2_cast54-align-cellTM-derive-bin.cnv")

# set up datetime to datenum 
ST_54 = pd.to_datetime(st54)

# split up and down casts; we want the downcast for the profile
down54, up54 = cast54.split()

# add a column with the start date and time in UTC (this is from the cnv header or from ship's log)
down54.insert(0, 'StartTime', pd.to_datetime(st54).replace(microsecond=0))

# slice the mixed layer out and save as csv
p2_54_ml = down54[down54['sigma-�00'].between(22.0,23.2)]
p2_54_ml.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/mixed_layer//Station20_cast54_ML.csv")

# get the sum of mixed layer slice oxygen, beam transmission, and chlorophyll fluor
cf_sum_p2_54_ml = p2_54_ml['flECO-AFL'].sum()
print('ML Chl fluor sum:', cf_sum_p2_54_ml)

bt_sum_p2_54_ml = p2_54_ml['CStarTr0'].sum()
print('ML Beam trans sum:', bt_sum_p2_54_ml)

o2_sum_p2_54_ml = p2_54_ml['sbeox0Mm/L'].sum()
print('ML O2 Mm/Kg sum:', o2_sum_p2_54_ml)

# Make lists of the input for the ML integrated dataframe
ML_data_54 = {'Cast':[54], 'StartTime':st54, 'StartTime matlab datenum': datetime_to_datenum(ST_54), \
           'Int chl fluor': cf_sum_p2_54_ml, \
           'Int beam trans': bt_sum_p2_54_ml, \
            'Int O2 Mm/Kg': o2_sum_p2_54_ml}

# slice the primary chlorophyll max out and save as csv
p2_54_pcm = down54[down54['sigma-�00'].between(23.2,54.7)]
p2_54_pcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/primary_chl/Station20_cast54_PCM.csv")

# get the sum of primary chl slice oxygen and beam transmission
cf_sum_p2_54_pcm = p2_54_pcm['flECO-AFL'].sum()
print('PCM Chl fluor sum:', cf_sum_p2_54_pcm)

bt_sum_p2_54_pcm = p2_54_pcm['CStarTr0'].sum()
print('PCM Beam trans sum:', bt_sum_p2_54_pcm)

o2_sum_p2_54_pcm = p2_54_pcm['sbeox0Mm/L'].sum()
print('PCM O2 Mm/Kg sum:', o2_sum_p2_54_pcm)

# Make lists of the input for the PCM integrated dataframe
PCM_data_54 = {'Cast':[54], 'StartTime':st54, 'StartTime matlab datenum': datetime_to_datenum(ST_54), \
            'Int chl fluor': cf_sum_p2_54_pcm, \
           'Int beam trans': bt_sum_p2_54_pcm, \
            'Int O2 Mm/Kg': o2_sum_p2_54_pcm}

# slice the deep chlorophyll max out and save as csv
p2_54_dcm = down54[down54['sigma-�00'].between(54.7,26.2)]
p2_54_dcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/deep_chl/Station20_cast54_DCM.csv")

# get the sum of deep chl slice oxygen and beam transmission
cf_sum_p2_54_dcm = p2_54_dcm['flECO-AFL'].sum()
print('DCM Chl fluor sum:', cf_sum_p2_54_dcm)

bt_sum_p2_54_dcm = p2_54_dcm['CStarTr0'].sum()
print('DCM Beam trans sum:', bt_sum_p2_54_dcm)

o2_sum_p2_54_dcm = p2_54_dcm['sbeox0Mm/L'].sum()
print('DCM O2 Mm/Kg sum:', o2_sum_p2_54_dcm)

# Make lists of the input for the DCM integrated dataframe
DCM_data_54 = {'Cast':[54], 'StartTime':st54, 'StartTime matlab datenum': datetime_to_datenum(ST_54), \
            'Int chl fluor': cf_sum_p2_54_dcm, \
           'Int beam trans': bt_sum_p2_54_dcm, \
            'Int O2 Mm/Kg': o2_sum_p2_54_dcm}

ML Chl fluor sum: 3.3330999999999995
ML Beam trans sum: 1077.6744
ML O2 Mm/Kg sum: 2451.654
PCM Chl fluor sum: 63.292699999999996
PCM Beam trans sum: 27045.731699999997
PCM O2 Mm/Kg sum: 9176.788
DCM Chl fluor sum: 0.0
DCM Beam trans sum: 0.0
DCM O2 Mm/Kg sum: 0.0


In [33]:
# read from cnv with 'ignore' added to depSM
cast55 = ctd.from_cnv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/KM1919-St20-P2/St20_P2_cast55-align-cellTM-derive-bin.cnv")

# set up datetime to datenum 
ST_55 = pd.to_datetime(st55)

# split up and down casts; we want the downcast for the profile
down55, up55 = cast55.split()

# add a column with the start date and time in UTC (this is from the cnv header or from ship's log)
down55.insert(0, 'StartTime', pd.to_datetime(st55).replace(microsecond=0))

# slice the mixed layer out and save as csv
p2_55_ml = down55[down55['sigma-�00'].between(22.0,23.2)]
p2_55_ml.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/mixed_layer//Station20_cast55_ML.csv")

# get the sum of mixed layer slice oxygen, beam transmission, and chlorophyll fluor
cf_sum_p2_55_ml = p2_55_ml['flECO-AFL'].sum()
print('ML Chl fluor sum:', cf_sum_p2_55_ml)

bt_sum_p2_55_ml = p2_55_ml['CStarTr0'].sum()
print('ML Beam trans sum:', bt_sum_p2_55_ml)

o2_sum_p2_55_ml = p2_55_ml['sbeox0Mm/L'].sum()
print('ML O2 Mm/Kg sum:', o2_sum_p2_55_ml)

# Make lists of the input for the ML integrated dataframe
ML_data_55 = {'Cast':[55], 'StartTime':st55, 'StartTime matlab datenum': datetime_to_datenum(ST_55), \
           'Int chl fluor': cf_sum_p2_55_ml, \
           'Int beam trans': bt_sum_p2_55_ml, \
            'Int O2 Mm/Kg': o2_sum_p2_55_ml}

# slice the primary chlorophyll max out and save as csv
p2_55_pcm = down55[down55['sigma-�00'].between(23.2,55.7)]
p2_55_pcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/primary_chl/Station20_cast55_PCM.csv")

# get the sum of primary chl slice oxygen and beam transmission
cf_sum_p2_55_pcm = p2_55_pcm['flECO-AFL'].sum()
print('PCM Chl fluor sum:', cf_sum_p2_55_pcm)

bt_sum_p2_55_pcm = p2_55_pcm['CStarTr0'].sum()
print('PCM Beam trans sum:', bt_sum_p2_55_pcm)

o2_sum_p2_55_pcm = p2_55_pcm['sbeox0Mm/L'].sum()
print('PCM O2 Mm/Kg sum:', o2_sum_p2_55_pcm)

# Make lists of the input for the PCM integrated dataframe
PCM_data_55 = {'Cast':[55], 'StartTime':st55, 'StartTime matlab datenum': datetime_to_datenum(ST_55), \
            'Int chl fluor': cf_sum_p2_55_pcm, \
           'Int beam trans': bt_sum_p2_55_pcm, \
            'Int O2 Mm/Kg': o2_sum_p2_55_pcm}

# slice the deep chlorophyll max out and save as csv
p2_55_dcm = down55[down55['sigma-�00'].between(55.7,26.2)]
p2_55_dcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/deep_chl/Station20_cast55_DCM.csv")

# get the sum of deep chl slice oxygen and beam transmission
cf_sum_p2_55_dcm = p2_55_dcm['flECO-AFL'].sum()
print('DCM Chl fluor sum:', cf_sum_p2_55_dcm)

bt_sum_p2_55_dcm = p2_55_dcm['CStarTr0'].sum()
print('DCM Beam trans sum:', bt_sum_p2_55_dcm)

o2_sum_p2_55_dcm = p2_55_dcm['sbeox0Mm/L'].sum()
print('DCM O2 Mm/Kg sum:', o2_sum_p2_55_dcm)

# Make lists of the input for the DCM integrated dataframe
DCM_data_55 = {'Cast':[55], 'StartTime':st55, 'StartTime matlab datenum': datetime_to_datenum(ST_55), \
            'Int chl fluor': cf_sum_p2_55_dcm, \
           'Int beam trans': bt_sum_p2_55_dcm, \
            'Int O2 Mm/Kg': o2_sum_p2_55_dcm}

ML Chl fluor sum: 3.3379000000000003
ML Beam trans sum: 1366.1670000000001
ML O2 Mm/Kg sum: 3102.1140000000005
PCM Chl fluor sum: 80.78309999999999
PCM Beam trans sum: 46189.49770000001
PCM O2 Mm/Kg sum: 9692.837
DCM Chl fluor sum: 0.0
DCM Beam trans sum: 0.0
DCM O2 Mm/Kg sum: 0.0


In [34]:
# read from cnv with 'ignore' added to depSM
cast56 = ctd.from_cnv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/KM1919-St20-P2/St20_P2_cast56-align-cellTM-derive-bin.cnv")

# set up datetime to datenum 
ST_56 = pd.to_datetime(st56)

# split up and down casts; we want the downcast for the profile
down56, up56 = cast56.split()

# add a column with the start date and time in UTC (this is from the cnv header or from ship's log)
down56.insert(0, 'StartTime', pd.to_datetime(st56).replace(microsecond=0))

# slice the mixed layer out and save as csv
p2_56_ml = down56[down56['sigma-�00'].between(22.0,23.2)]
p2_56_ml.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/mixed_layer//Station20_cast56_ML.csv")

# get the sum of mixed layer slice oxygen, beam transmission, and chlorophyll fluor
cf_sum_p2_56_ml = p2_56_ml['flECO-AFL'].sum()
print('ML Chl fluor sum:', cf_sum_p2_56_ml)

bt_sum_p2_56_ml = p2_56_ml['CStarTr0'].sum()
print('ML Beam trans sum:', bt_sum_p2_56_ml)

o2_sum_p2_56_ml = p2_56_ml['sbeox0Mm/L'].sum()
print('ML O2 Mm/Kg sum:', o2_sum_p2_56_ml)

# Make lists of the input for the ML integrated dataframe
ML_data_56 = {'Cast':[56], 'StartTime':st56, 'StartTime matlab datenum': datetime_to_datenum(ST_56), \
           'Int chl fluor': cf_sum_p2_56_ml, \
           'Int beam trans': bt_sum_p2_56_ml, \
            'Int O2 Mm/Kg': o2_sum_p2_56_ml}

# slice the primary chlorophyll max out and save as csv
p2_56_pcm = down56[down56['sigma-�00'].between(23.2,56.7)]
p2_56_pcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/primary_chl/Station20_cast56_PCM.csv")

# get the sum of primary chl slice oxygen and beam transmission
cf_sum_p2_56_pcm = p2_56_pcm['flECO-AFL'].sum()
print('PCM Chl fluor sum:', cf_sum_p2_56_pcm)

bt_sum_p2_56_pcm = p2_56_pcm['CStarTr0'].sum()
print('PCM Beam trans sum:', bt_sum_p2_56_pcm)

o2_sum_p2_56_pcm = p2_56_pcm['sbeox0Mm/L'].sum()
print('PCM O2 Mm/Kg sum:', o2_sum_p2_56_pcm)

# Make lists of the input for the PCM integrated dataframe
PCM_data_56 = {'Cast':[56], 'StartTime':st56, 'StartTime matlab datenum': datetime_to_datenum(ST_56), \
            'Int chl fluor': cf_sum_p2_56_pcm, \
           'Int beam trans': bt_sum_p2_56_pcm, \
            'Int O2 Mm/Kg': o2_sum_p2_56_pcm}

# slice the deep chlorophyll max out and save as csv
p2_56_dcm = down56[down56['sigma-�00'].between(56.7,26.2)]
p2_56_dcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/deep_chl/Station20_cast56_DCM.csv")

# get the sum of deep chl slice oxygen and beam transmission
cf_sum_p2_56_dcm = p2_56_dcm['flECO-AFL'].sum()
print('DCM Chl fluor sum:', cf_sum_p2_56_dcm)

bt_sum_p2_56_dcm = p2_56_dcm['CStarTr0'].sum()
print('DCM Beam trans sum:', bt_sum_p2_56_dcm)

o2_sum_p2_56_dcm = p2_56_dcm['sbeox0Mm/L'].sum()
print('DCM O2 Mm/Kg sum:', o2_sum_p2_56_dcm)

# Make lists of the input for the DCM integrated dataframe
DCM_data_56 = {'Cast':[56], 'StartTime':st56, 'StartTime matlab datenum': datetime_to_datenum(ST_56), \
            'Int chl fluor': cf_sum_p2_56_dcm, \
           'Int beam trans': bt_sum_p2_56_dcm, \
            'Int O2 Mm/Kg': o2_sum_p2_56_dcm}

ML Chl fluor sum: 3.0772
ML Beam trans sum: 1071.2458000000001
ML O2 Mm/Kg sum: 2441.642
PCM Chl fluor sum: 67.00150000000001
PCM Beam trans sum: 26992.783799999997
PCM O2 Mm/Kg sum: 9517.682
DCM Chl fluor sum: 0.0
DCM Beam trans sum: 0.0
DCM O2 Mm/Kg sum: 0.0


In [35]:
# read from cnv with 'ignore' added to depSM
cast57 = ctd.from_cnv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/KM1919-St20-P2/St20_P2_cast57-align-cellTM-derive-bin.cnv")

# set up datetime to datenum 
ST_57 = pd.to_datetime(st57)

# split up and down casts; we want the downcast for the profile
down57, up57 = cast57.split()

# add a column with the start date and time in UTC (this is from the cnv header or from ship's log)
down57.insert(0, 'StartTime', pd.to_datetime(st57).replace(microsecond=0))

# slice the mixed layer out and save as csv
p2_57_ml = down57[down57['sigma-�00'].between(22.0,23.2)]
p2_57_ml.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/mixed_layer//Station20_cast57_ML.csv")

# get the sum of mixed layer slice oxygen, beam transmission, and chlorophyll fluor
cf_sum_p2_57_ml = p2_57_ml['flECO-AFL'].sum()
print('ML Chl fluor sum:', cf_sum_p2_57_ml)

bt_sum_p2_57_ml = p2_57_ml['CStarTr0'].sum()
print('ML Beam trans sum:', bt_sum_p2_57_ml)

o2_sum_p2_57_ml = p2_57_ml['sbeox0Mm/L'].sum()
print('ML O2 Mm/Kg sum:', o2_sum_p2_57_ml)

# Make lists of the input for the ML integrated dataframe
ML_data_57 = {'Cast':[57], 'StartTime':st57, 'StartTime matlab datenum': datetime_to_datenum(ST_57), \
           'Int chl fluor': cf_sum_p2_57_ml, \
           'Int beam trans': bt_sum_p2_57_ml, \
            'Int O2 Mm/Kg': o2_sum_p2_57_ml}

# slice the primary chlorophyll max out and save as csv
p2_57_pcm = down57[down57['sigma-�00'].between(23.2,57.7)]
p2_57_pcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/primary_chl/Station20_cast57_PCM.csv")

# get the sum of primary chl slice oxygen and beam transmission
cf_sum_p2_57_pcm = p2_57_pcm['flECO-AFL'].sum()
print('PCM Chl fluor sum:', cf_sum_p2_57_pcm)

bt_sum_p2_57_pcm = p2_57_pcm['CStarTr0'].sum()
print('PCM Beam trans sum:', bt_sum_p2_57_pcm)

o2_sum_p2_57_pcm = p2_57_pcm['sbeox0Mm/L'].sum()
print('PCM O2 Mm/Kg sum:', o2_sum_p2_57_pcm)

# Make lists of the input for the PCM integrated dataframe
PCM_data_57 = {'Cast':[57], 'StartTime':st57, 'StartTime matlab datenum': datetime_to_datenum(ST_57), \
            'Int chl fluor': cf_sum_p2_57_pcm, \
           'Int beam trans': bt_sum_p2_57_pcm, \
            'Int O2 Mm/Kg': o2_sum_p2_57_pcm}

# slice the deep chlorophyll max out and save as csv
p2_57_dcm = down57[down57['sigma-�00'].between(57.7,26.2)]
p2_57_dcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/deep_chl/Station20_cast57_DCM.csv")

# get the sum of deep chl slice oxygen and beam transmission
cf_sum_p2_57_dcm = p2_57_dcm['flECO-AFL'].sum()
print('DCM Chl fluor sum:', cf_sum_p2_57_dcm)

bt_sum_p2_57_dcm = p2_57_dcm['CStarTr0'].sum()
print('DCM Beam trans sum:', bt_sum_p2_57_dcm)

o2_sum_p2_57_dcm = p2_57_dcm['sbeox0Mm/L'].sum()
print('DCM O2 Mm/Kg sum:', o2_sum_p2_57_dcm)

# Make lists of the input for the DCM integrated dataframe
DCM_data_57 = {'Cast':[57], 'StartTime':st57, 'StartTime matlab datenum': datetime_to_datenum(ST_57), \
            'Int chl fluor': cf_sum_p2_57_dcm, \
           'Int beam trans': bt_sum_p2_57_dcm, \
            'Int O2 Mm/Kg': o2_sum_p2_57_dcm}

ML Chl fluor sum: 3.3351
ML Beam trans sum: 1170.3635000000002
ML O2 Mm/Kg sum: 2641.6189999999997
PCM Chl fluor sum: 76.7948
PCM Beam trans sum: 30844.2801
PCM O2 Mm/Kg sum: 10730.192000000001
DCM Chl fluor sum: 0.0
DCM Beam trans sum: 0.0
DCM O2 Mm/Kg sum: 0.0


In [36]:
# read from cnv with 'ignore' added to depSM
cast58 = ctd.from_cnv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/KM1919-St20-P2/St20_P2_cast58-align-cellTM-derive-bin.cnv")

# set up datetime to datenum 
ST_58 = pd.to_datetime(st58)

# split up and down casts; we want the downcast for the profile
down58, up58 = cast58.split()

# add a column with the start date and time in UTC (this is from the cnv header or from ship's log)
down58.insert(0, 'StartTime', pd.to_datetime(st58).replace(microsecond=0))

# slice the mixed layer out and save as csv
p2_58_ml = down58[down58['sigma-�00'].between(22.0,23.2)]
p2_58_ml.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/mixed_layer//Station20_cast58_ML.csv")

# get the sum of mixed layer slice oxygen, beam transmission, and chlorophyll fluor
cf_sum_p2_58_ml = p2_58_ml['flECO-AFL'].sum()
print('ML Chl fluor sum:', cf_sum_p2_58_ml)

bt_sum_p2_58_ml = p2_58_ml['CStarTr0'].sum()
print('ML Beam trans sum:', bt_sum_p2_58_ml)

o2_sum_p2_58_ml = p2_58_ml['sbeox0Mm/L'].sum()
print('ML O2 Mm/Kg sum:', o2_sum_p2_58_ml)

# Make lists of the input for the ML integrated dataframe
ML_data_58 = {'Cast':[58], 'StartTime':st58, 'StartTime matlab datenum': datetime_to_datenum(ST_58), \
           'Int chl fluor': cf_sum_p2_58_ml, \
           'Int beam trans': bt_sum_p2_58_ml, \
            'Int O2 Mm/Kg': o2_sum_p2_58_ml}

# slice the primary chlorophyll max out and save as csv
p2_58_pcm = down58[down58['sigma-�00'].between(23.2,58.7)]
p2_58_pcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/primary_chl/Station20_cast58_PCM.csv")

# get the sum of primary chl slice oxygen and beam transmission
cf_sum_p2_58_pcm = p2_58_pcm['flECO-AFL'].sum()
print('PCM Chl fluor sum:', cf_sum_p2_58_pcm)

bt_sum_p2_58_pcm = p2_58_pcm['CStarTr0'].sum()
print('PCM Beam trans sum:', bt_sum_p2_58_pcm)

o2_sum_p2_58_pcm = p2_58_pcm['sbeox0Mm/L'].sum()
print('PCM O2 Mm/Kg sum:', o2_sum_p2_58_pcm)

# Make lists of the input for the PCM integrated dataframe
PCM_data_58 = {'Cast':[58], 'StartTime':st58, 'StartTime matlab datenum': datetime_to_datenum(ST_58), \
            'Int chl fluor': cf_sum_p2_58_pcm, \
           'Int beam trans': bt_sum_p2_58_pcm, \
            'Int O2 Mm/Kg': o2_sum_p2_58_pcm}

# slice the deep chlorophyll max out and save as csv
p2_58_dcm = down58[down58['sigma-�00'].between(58.7,26.2)]
p2_58_dcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/deep_chl/Station20_cast58_DCM.csv")

# get the sum of deep chl slice oxygen and beam transmission
cf_sum_p2_58_dcm = p2_58_dcm['flECO-AFL'].sum()
print('DCM Chl fluor sum:', cf_sum_p2_58_dcm)

bt_sum_p2_58_dcm = p2_58_dcm['CStarTr0'].sum()
print('DCM Beam trans sum:', bt_sum_p2_58_dcm)

o2_sum_p2_58_dcm = p2_58_dcm['sbeox0Mm/L'].sum()
print('DCM O2 Mm/Kg sum:', o2_sum_p2_58_dcm)

# Make lists of the input for the DCM integrated dataframe
DCM_data_58 = {'Cast':[58], 'StartTime':st58, 'StartTime matlab datenum': datetime_to_datenum(ST_58), \
            'Int chl fluor': cf_sum_p2_58_dcm, \
           'Int beam trans': bt_sum_p2_58_dcm, \
            'Int O2 Mm/Kg': o2_sum_p2_58_dcm}

ML Chl fluor sum: 5.444100000000001
ML Beam trans sum: 2148.1081999999997
ML O2 Mm/Kg sum: 4875.87
PCM Chl fluor sum: 56.169700000000006
PCM Beam trans sum: 15427.616399999999
PCM O2 Mm/Kg sum: 9085.055
DCM Chl fluor sum: 0.0
DCM Beam trans sum: 0.0
DCM O2 Mm/Kg sum: 0.0


In [37]:
# read from cnv with 'ignore' added to depSM
cast59 = ctd.from_cnv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/KM1919-St20-P2/St20_P2_cast59-align-cellTM-derive-bin.cnv")

# set up datetime to datenum 
ST_59 = pd.to_datetime(st59)

# split up and down casts; we want the downcast for the profile
down59, up59 = cast59.split()

# add a column with the start date and time in UTC (this is from the cnv header or from ship's log)
down59.insert(0, 'StartTime', pd.to_datetime(st59).replace(microsecond=0))

# slice the mixed layer out and save as csv
p2_59_ml = down59[down59['sigma-�00'].between(22.0,23.2)]
p2_59_ml.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/mixed_layer//Station20_cast59_ML.csv")

# get the sum of mixed layer slice oxygen, beam transmission, and chlorophyll fluor
cf_sum_p2_59_ml = p2_59_ml['flECO-AFL'].sum()
print('ML Chl fluor sum:', cf_sum_p2_59_ml)

bt_sum_p2_59_ml = p2_59_ml['CStarTr0'].sum()
print('ML Beam trans sum:', bt_sum_p2_59_ml)

o2_sum_p2_59_ml = p2_59_ml['sbeox0Mm/L'].sum()
print('ML O2 Mm/Kg sum:', o2_sum_p2_59_ml)

# Make lists of the input for the ML integrated dataframe
ML_data_59 = {'Cast':[59], 'StartTime':st59, 'StartTime matlab datenum': datetime_to_datenum(ST_59), \
           'Int chl fluor': cf_sum_p2_59_ml, \
           'Int beam trans': bt_sum_p2_59_ml, \
            'Int O2 Mm/Kg': o2_sum_p2_59_ml}

# slice the primary chlorophyll max out and save as csv
p2_59_pcm = down59[down59['sigma-�00'].between(23.2,59.7)]
p2_59_pcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/primary_chl/Station20_cast59_PCM.csv")

# get the sum of primary chl slice oxygen and beam transmission
cf_sum_p2_59_pcm = p2_59_pcm['flECO-AFL'].sum()
print('PCM Chl fluor sum:', cf_sum_p2_59_pcm)

bt_sum_p2_59_pcm = p2_59_pcm['CStarTr0'].sum()
print('PCM Beam trans sum:', bt_sum_p2_59_pcm)

o2_sum_p2_59_pcm = p2_59_pcm['sbeox0Mm/L'].sum()
print('PCM O2 Mm/Kg sum:', o2_sum_p2_59_pcm)

# Make lists of the input for the PCM integrated dataframe
PCM_data_59 = {'Cast':[59], 'StartTime':st59, 'StartTime matlab datenum': datetime_to_datenum(ST_59), \
            'Int chl fluor': cf_sum_p2_59_pcm, \
           'Int beam trans': bt_sum_p2_59_pcm, \
            'Int O2 Mm/Kg': o2_sum_p2_59_pcm}

# slice the deep chlorophyll max out and save as csv
p2_59_dcm = down59[down59['sigma-�00'].between(59.7,26.2)]
p2_59_dcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/deep_chl/Station20_cast59_DCM.csv")

# get the sum of deep chl slice oxygen and beam transmission
cf_sum_p2_59_dcm = p2_59_dcm['flECO-AFL'].sum()
print('DCM Chl fluor sum:', cf_sum_p2_59_dcm)

bt_sum_p2_59_dcm = p2_59_dcm['CStarTr0'].sum()
print('DCM Beam trans sum:', bt_sum_p2_59_dcm)

o2_sum_p2_59_dcm = p2_59_dcm['sbeox0Mm/L'].sum()
print('DCM O2 Mm/Kg sum:', o2_sum_p2_59_dcm)

# Make lists of the input for the DCM integrated dataframe
DCM_data_59 = {'Cast':[59], 'StartTime':st59, 'StartTime matlab datenum': datetime_to_datenum(ST_59), \
            'Int chl fluor': cf_sum_p2_59_dcm, \
           'Int beam trans': bt_sum_p2_59_dcm, \
            'Int O2 Mm/Kg': o2_sum_p2_59_dcm}

ML Chl fluor sum: 5.5899
ML Beam trans sum: 2053.5796
ML O2 Mm/Kg sum: 4622.223999999999
PCM Chl fluor sum: 73.77029999999999
PCM Beam trans sum: 34621.30189999999
PCM O2 Mm/Kg sum: 11008.696000000002
DCM Chl fluor sum: 0.0
DCM Beam trans sum: 0.0
DCM O2 Mm/Kg sum: 0.0


In [38]:
# read from cnv with 'ignore' added to depSM
cast60 = ctd.from_cnv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/KM1919-St20-P2/St20_P2_cast60-align-cellTM-derive-bin.cnv")

# set up datetime to datenum 
ST_60 = pd.to_datetime(st60)

# split up and down casts; we want the downcast for the profile
down60, up60 = cast60.split()

# add a column with the start date and time in UTC (this is from the cnv header or from ship's log)
down60.insert(0, 'StartTime', pd.to_datetime(st60).replace(microsecond=0))

# slice the mixed layer out and save as csv
p2_60_ml = down60[down60['sigma-�00'].between(22.0,23.2)]
p2_60_ml.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/mixed_layer//Station20_cast60_ML.csv")

# get the sum of mixed layer slice oxygen, beam transmission, and chlorophyll fluor
cf_sum_p2_60_ml = p2_60_ml['flECO-AFL'].sum()
print('ML Chl fluor sum:', cf_sum_p2_60_ml)

bt_sum_p2_60_ml = p2_60_ml['CStarTr0'].sum()
print('ML Beam trans sum:', bt_sum_p2_60_ml)

o2_sum_p2_60_ml = p2_60_ml['sbeox0Mm/L'].sum()
print('ML O2 Mm/Kg sum:', o2_sum_p2_60_ml)

# Make lists of the input for the ML integrated dataframe
ML_data_60 = {'Cast':[60], 'StartTime':st60, 'StartTime matlab datenum': datetime_to_datenum(ST_60), \
           'Int chl fluor': cf_sum_p2_60_ml, \
           'Int beam trans': bt_sum_p2_60_ml, \
            'Int O2 Mm/Kg': o2_sum_p2_60_ml}

# slice the primary chlorophyll max out and save as csv
p2_60_pcm = down60[down60['sigma-�00'].between(23.2,60.7)]
p2_60_pcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/primary_chl/Station20_cast60_PCM.csv")

# get the sum of primary chl slice oxygen and beam transmission
cf_sum_p2_60_pcm = p2_60_pcm['flECO-AFL'].sum()
print('PCM Chl fluor sum:', cf_sum_p2_60_pcm)

bt_sum_p2_60_pcm = p2_60_pcm['CStarTr0'].sum()
print('PCM Beam trans sum:', bt_sum_p2_60_pcm)

o2_sum_p2_60_pcm = p2_60_pcm['sbeox0Mm/L'].sum()
print('PCM O2 Mm/Kg sum:', o2_sum_p2_60_pcm)

# Make lists of the input for the PCM integrated dataframe
PCM_data_60 = {'Cast':[60], 'StartTime':st60, 'StartTime matlab datenum': datetime_to_datenum(ST_60), \
            'Int chl fluor': cf_sum_p2_60_pcm, \
           'Int beam trans': bt_sum_p2_60_pcm, \
            'Int O2 Mm/Kg': o2_sum_p2_60_pcm}

# slice the deep chlorophyll max out and save as csv
p2_60_dcm = down60[down60['sigma-�00'].between(60.7,26.2)]
p2_60_dcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/deep_chl/Station20_cast60_DCM.csv")

# get the sum of deep chl slice oxygen and beam transmission
cf_sum_p2_60_dcm = p2_60_dcm['flECO-AFL'].sum()
print('DCM Chl fluor sum:', cf_sum_p2_60_dcm)

bt_sum_p2_60_dcm = p2_60_dcm['CStarTr0'].sum()
print('DCM Beam trans sum:', bt_sum_p2_60_dcm)

o2_sum_p2_60_dcm = p2_60_dcm['sbeox0Mm/L'].sum()
print('DCM O2 Mm/Kg sum:', o2_sum_p2_60_dcm)

# Make lists of the input for the DCM integrated dataframe
DCM_data_60 = {'Cast':[60], 'StartTime':st60, 'StartTime matlab datenum': datetime_to_datenum(ST_60), \
            'Int chl fluor': cf_sum_p2_60_dcm, \
           'Int beam trans': bt_sum_p2_60_dcm, \
            'Int O2 Mm/Kg': o2_sum_p2_60_dcm}

ML Chl fluor sum: 3.2786999999999997
ML Beam trans sum: 1461.8122999999998
ML O2 Mm/Kg sum: 3319.914
PCM Chl fluor sum: 156.47719999999998
PCM Beam trans sum: 169551.0435
PCM O2 Mm/Kg sum: 38274.883
DCM Chl fluor sum: 0.0
DCM Beam trans sum: 0.0
DCM O2 Mm/Kg sum: 0.0


In [39]:
# read from cnv with 'ignore' added to depSM
cast61 = ctd.from_cnv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/KM1919-St20-P2/St20_P2_cast61-align-cellTM-derive-bin.cnv")

# set up datetime to datenum 
ST_61 = pd.to_datetime(st61)

# split up and down casts; we want the downcast for the profile
down61, up61 = cast61.split()

# add a column with the start date and time in UTC (this is from the cnv header or from ship's log)
down61.insert(0, 'StartTime', pd.to_datetime(st61).replace(microsecond=0))

# slice the mixed layer out and save as csv
p2_61_ml = down61[down61['sigma-�00'].between(22.0,23.2)]
p2_61_ml.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/mixed_layer//Station20_cast61_ML.csv")

# get the sum of mixed layer slice oxygen, beam transmission, and chlorophyll fluor
cf_sum_p2_61_ml = p2_61_ml['flECO-AFL'].sum()
print('ML Chl fluor sum:', cf_sum_p2_61_ml)

bt_sum_p2_61_ml = p2_61_ml['CStarTr0'].sum()
print('ML Beam trans sum:', bt_sum_p2_61_ml)

o2_sum_p2_61_ml = p2_61_ml['sbeox0Mm/L'].sum()
print('ML O2 Mm/Kg sum:', o2_sum_p2_61_ml)

# Make lists of the input for the ML integrated dataframe
ML_data_61 = {'Cast':[61], 'StartTime':st61, 'StartTime matlab datenum': datetime_to_datenum(ST_61), \
           'Int chl fluor': cf_sum_p2_61_ml, \
           'Int beam trans': bt_sum_p2_61_ml, \
            'Int O2 Mm/Kg': o2_sum_p2_61_ml}

# slice the primary chlorophyll max out and save as csv
p2_61_pcm = down61[down61['sigma-�00'].between(23.2,61.7)]
p2_61_pcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/primary_chl/Station20_cast61_PCM.csv")

# get the sum of primary chl slice oxygen and beam transmission
cf_sum_p2_61_pcm = p2_61_pcm['flECO-AFL'].sum()
print('PCM Chl fluor sum:', cf_sum_p2_61_pcm)

bt_sum_p2_61_pcm = p2_61_pcm['CStarTr0'].sum()
print('PCM Beam trans sum:', bt_sum_p2_61_pcm)

o2_sum_p2_61_pcm = p2_61_pcm['sbeox0Mm/L'].sum()
print('PCM O2 Mm/Kg sum:', o2_sum_p2_61_pcm)

# Make lists of the input for the PCM integrated dataframe
PCM_data_61 = {'Cast':[61], 'StartTime':st61, 'StartTime matlab datenum': datetime_to_datenum(ST_61), \
            'Int chl fluor': cf_sum_p2_61_pcm, \
           'Int beam trans': bt_sum_p2_61_pcm, \
            'Int O2 Mm/Kg': o2_sum_p2_61_pcm}

# slice the deep chlorophyll max out and save as csv
p2_61_dcm = down61[down61['sigma-�00'].between(25.8,26.2)]
p2_61_dcm.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/deep_chl/Station20_cast61_DCM.csv")

# get the sum of deep chl slice oxygen and beam transmission
cf_sum_p2_61_dcm = p2_61_dcm['flECO-AFL'].sum()
print('DCM Chl fluor sum:', cf_sum_p2_61_dcm)

bt_sum_p2_61_dcm = p2_61_dcm['CStarTr0'].sum()
print('DCM Beam trans sum:', bt_sum_p2_61_dcm)

o2_sum_p2_61_dcm = p2_61_dcm['sbeox0Mm/L'].sum()
print('DCM O2 Mm/Kg sum:', o2_sum_p2_61_dcm)

# Make lists of the input for the DCM integrated dataframe
DCM_data_61 = {'Cast':[61], 'StartTime':st61, 'StartTime matlab datenum': datetime_to_datenum(ST_61), \
            'Int chl fluor': cf_sum_p2_61_dcm, \
           'Int beam trans': bt_sum_p2_61_dcm, \
            'Int O2 Mm/Kg': o2_sum_p2_61_dcm}

ML Chl fluor sum: 2.8427
ML Beam trans sum: 1268.3547
ML O2 Mm/Kg sum: 2881.893
PCM Chl fluor sum: 89.993
PCM Beam trans sum: 65665.2249
PCM O2 Mm/Kg sum: 10783.863999999998
DCM Chl fluor sum: 7.760199999999999
DCM Beam trans sum: 2858.1971
DCM O2 Mm/Kg sum: 59.605000000000004


In [40]:
#ML_2019_p2.loc[len(ML_2019_p2)] = ML_data_43
ML_2019_p2 = ML_2019_p2.append(ML_data_25, ignore_index=True)
ML_2019_p2 = ML_2019_p2.append(ML_data_28, ignore_index=True)
ML_2019_p2 = ML_2019_p2.append(ML_data_29, ignore_index=True)
ML_2019_p2 = ML_2019_p2.append(ML_data_30, ignore_index=True)
ML_2019_p2 = ML_2019_p2.append(ML_data_31, ignore_index=True)
ML_2019_p2 = ML_2019_p2.append(ML_data_32, ignore_index=True)
ML_2019_p2 = ML_2019_p2.append(ML_data_33, ignore_index=True)
ML_2019_p2 = ML_2019_p2.append(ML_data_34, ignore_index=True)
ML_2019_p2 = ML_2019_p2.append(ML_data_35, ignore_index=True)
ML_2019_p2 = ML_2019_p2.append(ML_data_36, ignore_index=True)
ML_2019_p2 = ML_2019_p2.append(ML_data_37, ignore_index=True)
ML_2019_p2 = ML_2019_p2.append(ML_data_38, ignore_index=True)
ML_2019_p2 = ML_2019_p2.append(ML_data_39, ignore_index=True)
ML_2019_p2 = ML_2019_p2.append(ML_data_40, ignore_index=True)
ML_2019_p2 = ML_2019_p2.append(ML_data_42, ignore_index=True)
ML_2019_p2 = ML_2019_p2.append(ML_data_43, ignore_index=True)
ML_2019_p2 = ML_2019_p2.append(ML_data_44, ignore_index=True)
ML_2019_p2 = ML_2019_p2.append(ML_data_45, ignore_index=True)
ML_2019_p2 = ML_2019_p2.append(ML_data_46, ignore_index=True)
ML_2019_p2 = ML_2019_p2.append(ML_data_47, ignore_index=True)
ML_2019_p2 = ML_2019_p2.append(ML_data_48, ignore_index=True)
ML_2019_p2 = ML_2019_p2.append(ML_data_49, ignore_index=True)
ML_2019_p2 = ML_2019_p2.append(ML_data_50, ignore_index=True)
ML_2019_p2 = ML_2019_p2.append(ML_data_51, ignore_index=True)
ML_2019_p2 = ML_2019_p2.append(ML_data_52, ignore_index=True)
ML_2019_p2 = ML_2019_p2.append(ML_data_53, ignore_index=True)
ML_2019_p2 = ML_2019_p2.append(ML_data_54, ignore_index=True)
ML_2019_p2 = ML_2019_p2.append(ML_data_55, ignore_index=True)
ML_2019_p2 = ML_2019_p2.append(ML_data_56, ignore_index=True)
ML_2019_p2 = ML_2019_p2.append(ML_data_57, ignore_index=True)
ML_2019_p2 = ML_2019_p2.append(ML_data_58, ignore_index=True)
ML_2019_p2 = ML_2019_p2.append(ML_data_59, ignore_index=True)
ML_2019_p2 = ML_2019_p2.append(ML_data_60, ignore_index=True)
ML_2019_p2 = ML_2019_p2.append(ML_data_61, ignore_index=True)

ML_2019_p2.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/mixed_layer/Station20_integrated_ML.csv")

ML_2019_p2.head(16)

,Cast,StartTime,StartTime matlab datenum,Int chl fluor,Int beam trans,Int O2 Mm/Kg
0,25,2019-10-03 13:31:35,744916.563600,4.4364,1279.5397,2828.725
1,[25],2019-10-03 13:31:35,744916.563600,4.4364,1279.5397,2828.725
2,[28],2019-10-04 11:01:39,744918.459479,4.1782,1375.1804,3074.425
3,[29],2019-10-04 23:50:04,744918.993102,4.4235,1172.4138,2621.004
4,[30],2019-10-05 08:39:43,744920.360914,4.9495,1570.6556,3497.106
5,[31],2019-10-05 14:03:02,744920.585440,6.0353,1867.5779,4127.896
6,[32],2019-10-05 21:38:22,744920.901644,4.1007,1467.1161,3278.642
7,[33],2019-10-06 14:14:26,744922.593356,3.4269,1277.9139,2817.387
8,[34],2019-10-06 19:38:21,744922.818299,1.9979,879.6100,1945.229
9,[35],2019-10-06 22:48:57,744922.950660,3.8921,1366.4841,3034.705


In [41]:
#PCM_2019_p2.loc[len(PCM_2019_p2)] = PCM_data_43 

PCM_2019_p2 = PCM_2019_p2.append(PCM_data_25, ignore_index=True)
PCM_2019_p2 = PCM_2019_p2.append(PCM_data_28, ignore_index=True)
PCM_2019_p2 = PCM_2019_p2.append(PCM_data_29, ignore_index=True)
PCM_2019_p2 = PCM_2019_p2.append(PCM_data_30, ignore_index=True)
PCM_2019_p2 = PCM_2019_p2.append(PCM_data_31, ignore_index=True)
PCM_2019_p2 = PCM_2019_p2.append(PCM_data_32, ignore_index=True)
PCM_2019_p2 = PCM_2019_p2.append(PCM_data_33, ignore_index=True)
PCM_2019_p2 = PCM_2019_p2.append(PCM_data_34, ignore_index=True)
PCM_2019_p2 = PCM_2019_p2.append(PCM_data_35, ignore_index=True)
PCM_2019_p2 = PCM_2019_p2.append(PCM_data_36, ignore_index=True)
PCM_2019_p2 = PCM_2019_p2.append(PCM_data_37, ignore_index=True)
PCM_2019_p2 = PCM_2019_p2.append(PCM_data_38, ignore_index=True)
PCM_2019_p2 = PCM_2019_p2.append(PCM_data_39, ignore_index=True)
PCM_2019_p2 = PCM_2019_p2.append(PCM_data_40, ignore_index=True)
PCM_2019_p2 = PCM_2019_p2.append(PCM_data_42, ignore_index=True)
PCM_2019_p2 = PCM_2019_p2.append(PCM_data_43, ignore_index=True)
PCM_2019_p2 = PCM_2019_p2.append(PCM_data_44, ignore_index=True)
PCM_2019_p2 = PCM_2019_p2.append(PCM_data_45, ignore_index=True)
PCM_2019_p2 = PCM_2019_p2.append(PCM_data_46, ignore_index=True)
PCM_2019_p2 = PCM_2019_p2.append(PCM_data_47, ignore_index=True)
PCM_2019_p2 = PCM_2019_p2.append(PCM_data_48, ignore_index=True)
PCM_2019_p2 = PCM_2019_p2.append(PCM_data_49, ignore_index=True)
PCM_2019_p2 = PCM_2019_p2.append(PCM_data_50, ignore_index=True)
PCM_2019_p2 = PCM_2019_p2.append(PCM_data_51, ignore_index=True)
PCM_2019_p2 = PCM_2019_p2.append(PCM_data_52, ignore_index=True)
PCM_2019_p2 = PCM_2019_p2.append(PCM_data_53, ignore_index=True)
PCM_2019_p2 = PCM_2019_p2.append(PCM_data_54, ignore_index=True)
PCM_2019_p2 = PCM_2019_p2.append(PCM_data_55, ignore_index=True)
PCM_2019_p2 = PCM_2019_p2.append(PCM_data_56, ignore_index=True)
PCM_2019_p2 = PCM_2019_p2.append(PCM_data_57, ignore_index=True)
PCM_2019_p2 = PCM_2019_p2.append(PCM_data_58, ignore_index=True)
PCM_2019_p2 = PCM_2019_p2.append(PCM_data_59, ignore_index=True)
PCM_2019_p2 = PCM_2019_p2.append(PCM_data_60, ignore_index=True)
PCM_2019_p2 = PCM_2019_p2.append(PCM_data_61, ignore_index=True)

PCM_2019_p2.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/primary_chl/Station20_integrated_PCM.csv")

PCM_2019_p2.head(16)

,Cast,StartTime,StartTime matlab datenum,Int chl fluor,Int beam trans,Int O2 Mm/Kg
0,25,2019-10-03 13:31:35,744916.563600,24.7593,4051.8445,3690.297
1,[25],2019-10-03 13:31:35,744916.563600,24.7593,4051.8445,3690.297
2,[28],2019-10-04 11:01:39,744918.459479,57.7095,26594.2224,5969.360
3,[29],2019-10-04 23:50:04,744918.993102,66.5312,26773.4716,5158.425
4,[30],2019-10-05 08:39:43,744920.360914,107.7809,96023.5814,7054.908
5,[31],2019-10-05 14:03:02,744920.585440,100.4304,94117.0924,6224.446
6,[32],2019-10-05 21:38:22,744920.901644,83.6852,44836.6030,6322.003
7,[33],2019-10-06 14:14:26,744922.593356,61.2967,15347.4816,7461.636
8,[34],2019-10-06 19:38:21,744922.818299,89.3837,65347.5107,7943.816
9,[35],2019-10-06 22:48:57,744922.950660,118.5174,114689.1344,11533.641


In [44]:
#DCM_2019_p2.loc[len(DCM_2019_p2)] = DCM_data_43 

DCM_2019_p2 = DCM_2019_p2.append(DCM_data_25, ignore_index=True)
DCM_2019_p2 = DCM_2019_p2.append(DCM_data_28, ignore_index=True)
DCM_2019_p2 = DCM_2019_p2.append(DCM_data_29, ignore_index=True)
DCM_2019_p2 = DCM_2019_p2.append(DCM_data_30, ignore_index=True)
DCM_2019_p2 = DCM_2019_p2.append(DCM_data_31, ignore_index=True)
DCM_2019_p2 = DCM_2019_p2.append(DCM_data_32, ignore_index=True)
DCM_2019_p2 = DCM_2019_p2.append(DCM_data_33, ignore_index=True)
DCM_2019_p2 = DCM_2019_p2.append(DCM_data_34, ignore_index=True)
DCM_2019_p2 = DCM_2019_p2.append(DCM_data_35, ignore_index=True)
DCM_2019_p2 = DCM_2019_p2.append(DCM_data_36, ignore_index=True)
DCM_2019_p2 = DCM_2019_p2.append(DCM_data_37, ignore_index=True)
DCM_2019_p2 = DCM_2019_p2.append(DCM_data_38, ignore_index=True)
DCM_2019_p2 = DCM_2019_p2.append(DCM_data_39, ignore_index=True)
DCM_2019_p2 = DCM_2019_p2.append(DCM_data_40, ignore_index=True)
DCM_2019_p2 = DCM_2019_p2.append(DCM_data_42, ignore_index=True)
DCM_2019_p2 = DCM_2019_p2.append(DCM_data_43, ignore_index=True)
DCM_2019_p2 = DCM_2019_p2.append(DCM_data_44, ignore_index=True)
DCM_2019_p2 = DCM_2019_p2.append(DCM_data_45, ignore_index=True)
DCM_2019_p2 = DCM_2019_p2.append(DCM_data_46, ignore_index=True)
DCM_2019_p2 = DCM_2019_p2.append(DCM_data_47, ignore_index=True)
DCM_2019_p2 = DCM_2019_p2.append(DCM_data_48, ignore_index=True)
DCM_2019_p2 = DCM_2019_p2.append(DCM_data_49, ignore_index=True)
DCM_2019_p2 = DCM_2019_p2.append(DCM_data_50, ignore_index=True)
DCM_2019_p2 = DCM_2019_p2.append(DCM_data_51, ignore_index=True)
DCM_2019_p2 = DCM_2019_p2.append(DCM_data_52, ignore_index=True)
DCM_2019_p2 = DCM_2019_p2.append(DCM_data_53, ignore_index=True)
DCM_2019_p2 = DCM_2019_p2.append(DCM_data_54, ignore_index=True)
DCM_2019_p2 = DCM_2019_p2.append(DCM_data_55, ignore_index=True)
DCM_2019_p2 = DCM_2019_p2.append(DCM_data_56, ignore_index=True)
DCM_2019_p2 = DCM_2019_p2.append(DCM_data_57, ignore_index=True)
DCM_2019_p2 = DCM_2019_p2.append(DCM_data_58, ignore_index=True)
DCM_2019_p2 = DCM_2019_p2.append(DCM_data_59, ignore_index=True)
DCM_2019_p2 = DCM_2019_p2.append(DCM_data_60, ignore_index=True)
DCM_2019_p2 = DCM_2019_p2.append(DCM_data_61, ignore_index=True)

DCM_2019_p2.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/ctd-cnv/KM1919/P2_2019/deep_chl/Station20_integrated_DCM.csv")

DCM_2019_p2.head(35)

,Cast,StartTime,StartTime matlab datenum,Int chl fluor,Int beam trans,Int O2 Mm/Kg
0,25,2019-10-03 13:31:35,744916.563600,16.0851,4968.1428,77.584
1,[25],2019-10-03 13:31:35,744916.563600,16.0851,4968.1428,77.584
2,[28],2019-10-04 11:01:39,744918.459479,0.0000,0.0000,0.000
3,[29],2019-10-04 23:50:04,744918.993102,0.0000,0.0000,0.000
4,[30],2019-10-05 08:39:43,744920.360914,0.0000,0.0000,0.000
5,[31],2019-10-05 14:03:02,744920.585440,0.0000,0.0000,0.000
6,[32],2019-10-05 21:38:22,744920.901644,0.0000,0.0000,0.000
7,[33],2019-10-06 14:14:26,744922.593356,0.0000,0.0000,0.000
8,[34],2019-10-06 19:38:21,744922.818299,0.0000,0.0000,0.000
9,[35],2019-10-06 22:48:57,744922.950660,0.0000,0.0000,0.000
